# Preamble

In [1]:
import os
import sys, importlib
import pkg_resources
pkg_resources.require("numpy==1.26.1")
pkg_resources.require("pandas==1.4.2")
pkg_resources.require("seaborn==0.12.0")

import logging
import warnings
import time

from copy import deepcopy
from typing import Tuple

import numpy as np
import pandas as pd

import math
import scipy
from scipy.optimize import curve_fit
from scipy.interpolate import griddata

from statannotations.Annotator import Annotator

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.collections import PatchCollection
from matplotlib.patches import Rectangle
import matplotlib.font_manager
import seaborn as sns

import utils
from utils import circular_mean, circular_mean_length, circular_variance, circular_stddev

from restore_data import recover_data

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
from IPython.display import set_matplotlib_formats
%matplotlib inline

log = logging.getLogger(__name__)

In [3]:
from IPython.display import set_matplotlib_formats
%matplotlib inline

## Style

In [4]:
plt.style.context('seaborn-paper')
set_matplotlib_formats('svg')
plt.rcParams.update(plt.rcParamsDefault)

linewidth = 0.25
titlesize = 'medium'
labelsize = 'small'
ticksize = 'x-small'

markersize = 10
scattersize = 5

palette_stages='gist_heat'

plt.rcParams.update({
    'figure.figsize': [5.9/2, 5.9/2 * 2/3],

    'text.usetex': False,
    'font.size': 10,
    'font.family': 'sans-serif',
    'font.sans-serif': 'Helvetica',

    'figure.titlesize': titlesize,
    'legend.title_fontsize': labelsize,
    'legend.fontsize': ticksize,
    'axes.labelsize': labelsize,
    'xtick.labelsize': ticksize,
    'ytick.labelsize': ticksize,
    'ytick.labelsize': ticksize,

    'figure.autolayout': False,

    'axes.linewidth': linewidth,
    'xtick.major.width': linewidth,
    'xtick.minor.width': 0.8*linewidth,
    'ytick.major.width': linewidth,
    'ytick.minor.width': 0.8*linewidth,
    'grid.linewidth': linewidth,

    'patch.linewidth': linewidth,

    'lines.markersize': scattersize
})

/tmp/ipykernel_79459/677450911.py:2: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats('svg')


In [5]:
from matplotlib import font_manager
matplotlib.font_manager.findSystemFonts(fontpaths='/usr/share/fonts/truetype',
                                        fontext='ttf')
font_manager.findfont('Helvetica', fallback_to_default=False)

'/usr/share/fonts/truetype/helvetica/Helvetica.ttf'

In [6]:
lineplot_kwargs = dict(
    markers=True,
    ls='',
    dashes=False,
    ci='sd',
    err_style='bars'
)

position_ordering = ["0", "25I", "25S", "50I", "50S", "75I", "75S", "100"]
# stage_ordering = ["E8", "E10", "E12", "E13", "E14"]
stage_ordering = ["E8", "E10", "E12", "E14"]

experiment_order=[
    'MLCKdmso-4hrs',
    'MLCKdmso-16hrs',
    'MLCKml7-4hrs',
    'MLCKml7-16hrs',
    'Vangl2Control',
    'Vangl2KO',
]

treatment_order=[
    'MLCKdmso',
    'MLCKml7',
    'Vangl2Control',
    'Vangl2KO',
    'LiveControl',
    'LiveML7',
]

lineplot_position_stage_kwargs = dict(
    style='position',
    style_order=position_ordering,
    hue='stage',
    hue_order=stage_ordering,
    size='sample_id',
    palette=palette_stages
)
lineplot_position_stage_kwargs.update(lineplot_kwargs)

## Import data

In [7]:
basepath = os.path.dirname('')
basepath = os.path.abspath(os.path.join(basepath, ".."))
data_basedir = "data"
rel_paths=[
    'HCA-Actn4_new/E8',
    'HCA-Actn4_new/E10',
    'HCA-Actn4_new/E12',
    'HCA-Actn4/E8',
    'HCA-Actn4/E10',
    'HCA-Actn4/E12',
    'HCA-Actn4/E14',
    'Vangl2_KO/Control',
    'Vangl2_KO/Treatment',
    'Vangl2_KO/Treatment_Vangl2',
    'Atoh1/Control',
    'Atoh1/Treatment',
    'MLCK_inhibition/DMSO__4hrs',
    'MLCK_inhibition/DMSO__16hrs',
    'MLCK_inhibition/ML7__4hrs',
    'MLCK_inhibition/ML7__16hrs',
    'ROCK_inhibitions',
    'PTX_inhibitions'
]
data_path = "102_treated_data"
save_file_prefix='summary'

In [8]:
figure_path = 'analysis/1_figures'
figure_path = os.path.abspath(os.path.join(basepath, figure_path))

In [9]:
paths = []
for p in rel_paths:
    paths.append(os.path.join(basepath, data_basedir, p, data_path))
cells, bonds = recover_data(paths=paths, save_file_prefix=save_file_prefix)
cells = cells.loc[~cells['is_border_cell']]
if not 'aligned_with_PD_axis' in cells.columns:
    cells['aligned_with_PD_axis'] = False
    bonds['aligned_with_PD_axis'] = False
else:
    cells['aligned_with_PD_axis'] = np.where(
        ~np.isnan(np.asarray(cells['aligned_with_PD_axis'], dtype=float)),
        cells['aligned_with_PD_axis'],
        False
    )
    bonds['aligned_with_PD_axis'] = np.where(
        ~np.isnan(np.asarray(bonds['aligned_with_PD_axis'], dtype=float)),
        bonds['aligned_with_PD_axis'],
        False
    )

if 'Experiment' in cells.columns:
    cells['Experiment'] = cells['Experiment'].where(
        ~cells['Experiment'].isnull(),
        'Control'
    )
    bonds['Experiment'] = bonds['Experiment'].where(
        ~bonds['Experiment'].isnull(),
        'Control'
    )
else:
    cells['Experiment'] = 'Control'
    bonds['Experiment'] = 'Control'

print(cells['Experiment'].unique())

Gathered 53427 cells, of which 12404 HC at stages ['E8' 'E10' 'E12' 'E14'] and positions ['25S' '25I' '50S' '50I' '75I' '0' '75S' '100'] from 104 files: 
 ['E8_25S_1_0.tif' 'E8_25I_1_2.tif' 'E8_25I_2_1.tif' 'E8_25I_3_2.tif'
 'E8_25I_1_0.tif' 'E8_50S_2_1.tif' 'E8_50I_1_1.tif' 'E8_50I_3_1.tif'
 'E8_50I_2_1.tif' 'E8_75I_1_1.tif' 'E8_75I_3_2.tif' 'E8_75I_2_1.tif'
 'E10_0_1_0.tif' 'E10_25S_2_0.tif' 'E10_25S_3_0.tif' 'E10_25I_1_0.tif'
 'E10_25I_2_0.tif' 'E10_25I_1_1.tif' 'E10_25I_3_1.tif'
 'E10_Vangl2Control_1.tif' 'E10_Vangl2Control_2.tif'
 'E10_Vangl2Control_3.tif' 'E10_Vangl2KO_1.tif' 'E10_Vangl2KO_2.tif'
 'E10_Vangl2KO_3.tif' 'Vangl2KO_1.tif' 'Vangl2KO_2.tif' 'Vangl2KO_3.tif'
 'Vangl2KO_4.tif' 'Atoh1control_1.tif' 'Atoh1control_2.tif'
 'Atoh1control_3.tif' 'Atoh1control_4.tif' 'Atoh1KO_1.tif' 'Atoh1KO_2.tif'
 'Atoh1KO_3.tif' 'Atoh1KO_4.tif' 'MLCKdmso-4hrs_1.tif'
 'MLCKdmso-4hrs_2.tif' 'MLCKdmso-4hrs_3.tif' 'MLCKdmso-16hrs_1.tif'
 'MLCKdmso-16hrs_2.tif' 'MLCKdmso-16hrs_3.tif' 'MLCKml7-4hr

In [10]:


# Map to treatment
cells['Treatment'] = cells['Experiment']
bonds['Treatment'] = bonds['Experiment']

cells.loc[cells['Experiment']=='Live0Control', 'Treatment'] = 'LiveControl'
cells.loc[cells['Experiment']=='Live150Control', 'Treatment'] = 'LiveControl'
cells.loc[cells['Experiment']=='Live480Control', 'Treatment'] = 'LiveControl'
cells.loc[cells['Experiment']=='Live0ML7', 'Treatment'] = 'LiveML7'
cells.loc[cells['Experiment']=='Live150ML7', 'Treatment'] = 'LiveML7'
cells.loc[cells['Experiment']=='Live400ML7', 'Treatment'] = 'LiveML7'

bonds.loc[bonds['Experiment']=='Live0Control', 'Treatment'] = 'LiveControl'
bonds.loc[bonds['Experiment']=='Live150Control', 'Treatment'] = 'LiveControl'
bonds.loc[bonds['Experiment']=='Live480Control', 'Treatment'] = 'LiveControl'
bonds.loc[bonds['Experiment']=='Live0ML7', 'Treatment'] = 'LiveML7'
bonds.loc[bonds['Experiment']=='Live150ML7', 'Treatment'] = 'LiveML7'
bonds.loc[bonds['Experiment']=='Live400ML7', 'Treatment'] = 'LiveML7'

cells['Experiment_Series'] = cells['Treatment']
cells.loc[cells['Treatment']=='Atoh1control', 'Experiment_Series'] = 'Atoh1'
cells.loc[cells['Treatment']=='Atoh1KO', 'Experiment_Series'] = 'Atoh1'
cells.loc[cells['Treatment']=='MLCKdmso-4hrs', 'Experiment_Series'] = 'MLCK'
cells.loc[cells['Treatment']=='MLCKml7-4hrs', 'Experiment_Series'] = 'MLCK'
cells.loc[cells['Treatment']=='MLCKdmso-16hrs', 'Experiment_Series'] = 'MLCK_16hrs'
cells.loc[cells['Treatment']=='MLCKml7-16hrs', 'Experiment_Series'] = 'MLCK_16hrs'
cells.loc[cells['Treatment']=='ROCKcontrol-4hrs', 'Experiment_Series'] = 'ROCK'
cells.loc[cells['Treatment']=='ROCKy27632-4hrs', 'Experiment_Series'] = 'ROCK'
cells.loc[cells['Treatment']=='ROCKcontrol-16hrs', 'Experiment_Series'] = 'ROCK_16hrs'
cells.loc[cells['Treatment']=='ROCKy27632-16hrs', 'Experiment_Series'] = 'ROCK_16hrs'
cells.loc[cells['Treatment']=='PTXcontrol', 'Experiment_Series'] = 'PTX'
cells.loc[cells['Treatment']=='PTXinhibition', 'Experiment_Series'] = 'PTX'
cells.loc[cells['Treatment']=='Vangl2Control', 'Experiment_Series'] = 'Vangl2'
cells.loc[cells['Treatment']=='Vangl2KO', 'Experiment_Series'] = 'Vangl2'
cells.loc[cells['Treatment']=='LiveControl', 'Experiment_Series'] = 'Live'
cells.loc[cells['Treatment']=='LiveML7', 'Experiment_Series'] = 'Live'

bonds['Experiment_Series'] = bonds['Treatment']
bonds.loc[bonds['Treatment']=='Atoh1control', 'Experiment_Series'] = 'Atoh1'
bonds.loc[bonds['Treatment']=='Atoh1KO', 'Experiment_Series'] = 'Atoh1'
bonds.loc[bonds['Treatment']=='MLCKdmso-4hrs', 'Experiment_Series'] = 'MLCK'
bonds.loc[bonds['Treatment']=='MLCKml7-4hrs', 'Experiment_Series'] = 'MLCK'
bonds.loc[bonds['Treatment']=='MLCKdmso-16hrs', 'Experiment_Series'] = 'MLCK_16hrs'
bonds.loc[bonds['Treatment']=='MLCKml7-16hrs', 'Experiment_Series'] = 'MLCK_16hrs'
bonds.loc[bonds['Treatment']=='ROCKcontrol-4hrs', 'Experiment_Series'] = 'ROCK'
bonds.loc[bonds['Treatment']=='ROCKy27632-4hrs', 'Experiment_Series'] = 'ROCK'
bonds.loc[bonds['Treatment']=='ROCKcontrol-16hrs', 'Experiment_Series'] = 'ROCK_16hrs'
bonds.loc[bonds['Treatment']=='ROCKy27632-16hrs', 'Experiment_Series'] = 'ROCK_16hrs'
bonds.loc[bonds['Treatment']=='PTXcontrol', 'Experiment_Series'] = 'PTX'
bonds.loc[bonds['Treatment']=='PTXinhibition', 'Experiment_Series'] = 'PTX'
bonds.loc[bonds['Treatment']=='Vangl2Control', 'Experiment_Series'] = 'Vangl2'
bonds.loc[bonds['Treatment']=='Vangl2KO', 'Experiment_Series'] = 'Vangl2'
bonds.loc[bonds['Treatment']=='LiveControl', 'Experiment_Series'] = 'Live'
bonds.loc[bonds['Treatment']=='LiveML7', 'Experiment_Series'] = 'Live'

{figure_path}
# Assign is_control
cells['is_Control'] = False
cells.loc[cells['Treatment']=='Control', 'is_Control'] = True
cells.loc[cells['Treatment']=='Atoh1control', 'is_Control'] = True
cells.loc[cells['Treatment']=='Atoh1KO', 'is_Control'] = False
cells.loc[cells['Treatment']=='MLCKdmso-4hrs', 'is_Control'] = True
cells.loc[cells['Treatment']=='MLCKml7-4hrs', 'is_Control'] = False
cells.loc[cells['Treatment']=='MLCKdmso-16hrs', 'is_Control'] = True
cells.loc[cells['Treatment']=='MLCKml7-16hrs', 'is_Control'] = False
cells.loc[cells['Treatment']=='ROCKcontrol-4hrs', 'is_Control'] = True
cells.loc[cells['Treatment']=='ROCKy27632-4hrs', 'is_Control'] = False
cells.loc[cells['Treatment']=='ROCKcontrol-16hrs', 'is_Control'] = True
cells.loc[cells['Treatment']=='ROCKy27632-16hrs', 'is_Control'] = False
cells.loc[cells['Treatment']=='PTXcontrol', 'is_Control'] = True
cells.loc[cells['Treatment']=='PTXinhibition', 'is_Control'] = False
cells.loc[cells['Treatment']=='Vangl2Control', 'is_Control'] = True
cells.loc[cells['Treatment']=='Vangl2KO', 'is_Control'] = False
cells.loc[cells['Treatment']=='LiveControl', 'is_Control'] = True
cells.loc[cells['Treatment']=='LiveML7', 'is_Control'] = False

bonds['is_Control'] = False
bonds.loc[bonds['Treatment']=='Control', 'is_Control'] = True
bonds.loc[bonds['Treatment']=='Atoh1control', 'is_Control'] = True
bonds.loc[bonds['Treatment']=='Atoh1KO', 'is_Control'] = False
bonds.loc[bonds['Treatment']=='MLCKdmso-4hrs', 'is_Control'] = True
bonds.loc[bonds['Treatment']=='MLCKml7-4hrs', 'is_Control'] = False
bonds.loc[bonds['Treatment']=='MLCKdmso-16hrs', 'is_Control'] = True
bonds.loc[bonds['Treatment']=='MLCKml7-16hrs', 'is_Control'] = False
bonds.loc[bonds['Treatment']=='ROCKcontrol-4hrs', 'is_Control'] = True
bonds.loc[bonds['Treatment']=='ROCKy27632-4hrs', 'is_Control'] = False
bonds.loc[bonds['Treatment']=='ROCKcontrol-16hrs', 'is_Control'] = True
bonds.loc[bonds['Treatment']=='ROCKy27632-16hrs', 'is_Control'] = False
bonds.loc[bonds['Treatment']=='PTXcontrol', 'is_Control'] = True
bonds.loc[bonds['Treatment']=='PTXinhibition', 'is_Control'] = False
bonds.loc[bonds['Treatment']=='Vangl2Control', 'is_Control'] = True
bonds.loc[bonds['Treatment']=='Vangl2KO', 'is_Control'] = False
bonds.loc[bonds['Treatment']=='LiveControl', 'is_Control'] = True
bonds.loc[bonds['Treatment']=='LiveML7', 'is_Control'] = False

### Correct orientations

In [11]:
# correct orientations
print("WARNING reorienting cilia orientations, but not other orientations")
mask_rotate90 = (
    (cells['filename'] == 'MLCKdmso-4hrs_2.tif')
)
cells.loc[mask_rotate90, 'cilium_phi'] -= math.pi/2
cells.loc[mask_rotate90, 'cilium_phi_corrected'] -= math.pi/2

mask_flip_SI = (
    (cells['Experiment'] == 'MLCKdmso-4hrs') |
    (
        (cells['Experiment'] == 'Control') &
        (cells['stage'] == 'E14') &
        (cells['position'] == '75I')
    )
)
print(mask_flip_SI.sum())
cells.loc[mask_flip_SI, 'cilium_phi'] *= -1
cells.loc[mask_flip_SI, 'cilium_phi_corrected'] *= -1


# correct y-axis pointing down
cells['cilium_phi'] *= -1
cells['cilium_phi_corrected'] *= -1


cells['cilium_DX'] = cells['cilium_rho'] * np.cos(cells['cilium_phi'])
cells['cilium_DY'] = cells['cilium_rho'] * np.sin(cells['cilium_phi'])
cells['cilium_DX_corrected'] = cells['cilium_rho_corrected'] * np.cos(cells['cilium_phi_corrected'])
cells['cilium_DY_corrected'] = cells['cilium_rho_corrected'] * np.sin(cells['cilium_phi_corrected'])

WARNING reorienting cilia orientations, but not other orientations
2159


### Corrected neighbourhood

In [12]:
# Cells by type in Ex-ovo data
HC = cells.loc[(cells['is_HC'])  & (cells['Experiment'] == 'Control')]
SC = cells.loc[(~cells['is_HC']) & (cells['Experiment'] == 'Control')]

# Linear fit neighbour number to cell area
m_HC = np.polyfit(HC['normalized_area_cells'], HC['num_neighbors'], 1)[0]
m_SC = np.polyfit(SC['normalized_area_cells'], SC['num_neighbors'], 1)[0]

cells['num_neighbors__corr'] = np.where(
    cells['is_HC'],
    cells['num_neighbors'] + m_HC * (1 - cells['normalized_area_cells']),
    cells['num_neighbors'] + m_SC * (1 - cells['normalized_area_cells']),
)

HC = None
SC = None
del HC, SC

### Align tissue axes

In [13]:
# Average orientations per image
grp=[
    'Experiment_Series',
    'stage',
    'position',
    'is_Control',
    'filename',
    'file_id'
]
grp_data = cells.copy().groupby(by=grp)

p_vec = ['cilium_DX_corrected', 'cilium_DY_corrected']

results = pd.DataFrame()
results['mean_orientation'] = grp_data[p_vec].apply(circular_mean, eulerian=True)
results['circular_stddev'] = grp_data[p_vec].apply(circular_stddev, eulerian=True)
results['circular_variance'] = grp_data[p_vec].apply(circular_variance, eulerian=True)
results['resulting_length'] = grp_data[p_vec].apply(circular_mean_length, eulerian=True)
results['datapoints'] = grp_data['cilium_DX_corrected'].apply(lambda d: d.count())
results['aligned_with_PD_axis'] = grp_data['aligned_with_PD_axis'].all()

results['mean_orientation__angles'] = (results['mean_orientation'] / math.pi * 180 + 360) % 360
results['circular_stddev__angles'] = (results['circular_stddev'] / math.pi * 180 + 360) % 360

results.to_csv("{}/{}/{}_polarity.csv".format(basepath, data_basedir, save_file_prefix))
results = results.reset_index()

grp_data = None
del grp_data

In [14]:
# align cilium_phi by circular mean of mean orientation
cells['cilium_phi_corrected__aligned'] = cells['cilium_phi_corrected'].copy()
for file_id in cells['file_id'].unique():
    cells.loc[cells['file_id']==file_id, 'cilium_phi_corrected__aligned'] -= results.loc[results['file_id']==file_id, 'mean_orientation'].iloc[0]

for series in results['Experiment_Series'].unique():
    for is_Control in results['is_Control'].unique():
        mask = (
            (results['Experiment_Series']==series) &
            (results['is_Control']==is_Control) &
            (results['aligned_with_PD_axis'])
        )

        cmean = circular_mean(
            results.loc[mask, ['resulting_length', 'mean_orientation']],
            eulerian=False
        )

        mask = (
            (cells['Experiment_Series']==series) &
            (cells['is_Control']==is_Control)
        )
        cells.loc[mask, 'cilium_phi_corrected__aligned'] += cmean

cells['cilium_DX_corrected__aligned'] = cells['cilium_rho_corrected'] * np.cos(cells['cilium_phi_corrected__aligned'])
cells['cilium_DY_corrected__aligned'] = cells['cilium_rho_corrected'] * np.sin(cells['cilium_phi_corrected__aligned'])


# center aligned cilium_phi to inferior (90 deg) orientation
cells['cilium_phi_corrected__centred'] = cells['cilium_phi_corrected'].copy()
for file_id in cells['file_id'].unique():
    cells.loc[cells['file_id']==file_id, 'cilium_phi_corrected__centred'] -= results.loc[results['file_id']==file_id, 'mean_orientation'].iloc[0] + math.pi/2.

cells['cilium_DX_corrected__centred'] = cells['cilium_rho_corrected'] * np.cos(cells['cilium_phi_corrected__centred'])
cells['cilium_DY_corrected__centred'] = cells['cilium_rho_corrected'] * np.sin(cells['cilium_phi_corrected__centred'])


/home/julian/image_analysis/BP_segmentation/utils.py:147: RuntimeWarning: Mean of empty slice.
  mean_x = x.mean()
/home/julian/.local/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/home/julian/image_analysis/BP_segmentation/utils.py:148: RuntimeWarning: Mean of empty slice.
  mean_y = y.mean()
/home/julian/image_analysis/BP_segmentation/utils.py:147: RuntimeWarning: Mean of empty slice.
  mean_x = x.mean()
/home/julian/.local/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/home/julian/image_analysis/BP_segmentation/utils.py:148: RuntimeWarning: Mean of empty slice.
  mean_y = y.mean()
/home/julian/image_analysis/BP_segmentation/utils.py:147: RuntimeWarning: Mean of empty slice.
  mean_x = x.mean()
/home/julian/.local/lib/python3.10/site-packages/numpy/core/_methods.py:129: Ru

In [15]:
# Average orientations per image type
grp=[
    'Experiment_Series',
    'stage',
    'position',
    'is_Control',
]
grp_data = cells.copy().groupby(by=grp)

p_vec = ['cilium_DX_corrected', 'cilium_DY_corrected']


results_global_mean = pd.DataFrame()
grp_global_mean=grp[:-2]
results_global_mean['mean_orientation'] = grp_data[p_vec].apply(circular_mean, eulerian=True)
results_global_mean['circular_stddev'] = grp_data[p_vec].apply(circular_stddev, eulerian=True)
results_global_mean['circular_variance'] = grp_data[p_vec].apply(circular_variance, eulerian=True)
results_global_mean['resulting_length'] = grp_data[p_vec].apply(circular_mean_length, eulerian=True)
results_global_mean['datapoints'] = grp_data['cilium_DX_corrected'].apply(lambda d: d.count())

results_global_mean['mean_orientation__angles'] = (results_global_mean['mean_orientation'] / math.pi * 180 + 360) % 360
results_global_mean['circular_stddev__angles'] = (results_global_mean['circular_stddev'] / math.pi * 180 + 360) % 360

results_global_mean.to_csv("{}/{}/{}_polarity__global_mean.csv".format(basepath, data_basedir, save_file_prefix))

grp_data = None
results_global_mean = None
del grp_data, results_global_mean

In [16]:
# Average orientations in aligned images

grp_data = cells.copy().groupby(by=grp)

p_vec = ['cilium_DX_corrected__aligned', 'cilium_DY_corrected__aligned']

results_aligned = pd.DataFrame()
results_aligned['mean_orientation'] = grp_data[p_vec].apply(circular_mean, eulerian=True)
results_aligned['circular_stddev'] = grp_data[p_vec].apply(circular_stddev, eulerian=True)
results_aligned['circular_variance'] = grp_data[p_vec].apply(circular_variance, eulerian=True)
results_aligned['resulting_length'] = grp_data[p_vec].apply(circular_mean_length, eulerian=True)
results_aligned['datapoints'] = grp_data['cilium_DX_corrected__aligned'].apply(lambda d: d.count())

results_aligned['mean_orientation__angles'] = (results_aligned['mean_orientation'] / math.pi * 180 + 360) % 360
results_aligned['circular_stddev__angles'] = (results_aligned['circular_stddev'] / math.pi * 180 + 360) % 360

results_aligned.to_csv("{}/{}/{}_polarity__aligned.csv".format(basepath, data_basedir, save_file_prefix))

grp_data = None
results_aligned = None
del grp_data, results_aligned

In [17]:
results = None
del results

### Tissue avearges

In [18]:
def tissue_averages(cells, bonds, tissue_grp):
    """Calculates numerous average quantities based on the tissue_grp groups
    """
    cells = cells.loc[~cells['is_border_cell']]
    HC = cells.loc[cells['is_HC']].copy()
    SC = cells.loc[~cells['is_HC']].copy()

    tissue_data = pd.DataFrame()

    # count cells
    tissue_data['num_HC'] = HC.groupby(by=tissue_grp)['is_HC'].count()
    tissue_data['num_SC'] = SC.groupby(by=tissue_grp)['is_HC'].count()
    tissue_data['num_cells'] = cells.groupby(by=tissue_grp)['is_HC'].count()

    tissue_data['HC_density'] = tissue_data['num_HC'] / tissue_data['num_cells']
    tissue_data['number_ratio'] = tissue_data['num_HC'] / tissue_data['num_SC']

    # cell areas
    tissue_data['HC_area'] = HC.groupby(by=tissue_grp)['normalized_area_cells'].mean()
    tissue_data['HC_area__std'] = HC.groupby(by=tissue_grp)['normalized_area_cells'].std()

    tissue_data['SC_area'] = SC.groupby(by=tissue_grp)['normalized_area_cells'].mean()
    tissue_data['SC_area__std'] = SC.groupby(by=tissue_grp)['normalized_area_cells'].std()

    tissue_data['area__no_norm'] = cells.groupby(by=tissue_grp)['area'].mean()
    tissue_data['area__no_norm__std'] = cells.groupby(by=tissue_grp)['area'].std()

    tissue_data['HC_area__no_norm'] = HC.groupby(by=tissue_grp)['area'].mean()
    tissue_data['HC_area__no_norm__std'] = HC.groupby(by=tissue_grp)['area'].std()

    tissue_data['SC_area__no_norm'] = SC.groupby(by=tissue_grp)['area'].mean()
    tissue_data['SC_area__no_norm__std'] = SC.groupby(by=tissue_grp)['area'].std()


    tissue_data['relative_area'] = (
        HC.groupby(by=tissue_grp)['normalized_area_cells'].mean()
        / SC.groupby(by=tissue_grp)['normalized_area_cells'].mean()
    )
    tissue_data['relative_area__std'] = tissue_data['relative_area'] * abs(
        (  HC.groupby(by=tissue_grp)['normalized_area_cells'].std()
        / HC.groupby(by=tissue_grp)['normalized_area_cells'].mean())
        -
        (  SC.groupby(by=tissue_grp)['normalized_area_cells'].std()
        / SC.groupby(by=tissue_grp)['normalized_area_cells'].mean())
    )
    tissue_data['HC_area_coverage'] = tissue_data['HC_area'] * tissue_data['HC_density']


    # neighborhood
    tissue_data['neighbor_number'] = cells.groupby(by=tissue_grp)['num_neighbors'].mean()
    tissue_data['neighbor_number__std'] = cells.groupby(by=tissue_grp)['num_neighbors'].std()
    tissue_data['neighbor_number__corr'] = cells.groupby(by=tissue_grp)['num_neighbors__corr'].mean()
    tissue_data['neighbor_number__corr__std'] = cells.groupby(by=tissue_grp)['num_neighbors__corr'].std()

    tissue_data['coordination_number'] = HC.groupby(by=tissue_grp)['num_neighbors'].mean()
    tissue_data['coordination_number__std'] = HC.groupby(by=tissue_grp)['num_neighbors'].std()
    tissue_data['coordination_number__corr'] = HC.groupby(by=tissue_grp)['num_neighbors__corr'].mean()
    tissue_data['coordination_number__corr__std'] = HC.groupby(by=tissue_grp)['num_neighbors__corr'].std()

    tissue_data['subordination_number'] = SC.groupby(by=tissue_grp)['num_neighbors'].mean()
    tissue_data['subordination_number__std'] = SC.groupby(by=tissue_grp)['num_neighbors'].std()
    tissue_data['subordination_number__corr'] = SC.groupby(by=tissue_grp)['num_neighbors__corr'].mean()
    tissue_data['subordination_number__corr__std'] = SC.groupby(by=tissue_grp)['num_neighbors__corr'].std()

    tissue_data['HH_contacts'] = HC.groupby(by=tissue_grp)['num_hair_neighbors'].mean()
    tissue_data['HH_contacts__std'] = HC.groupby(by=tissue_grp)['num_hair_neighbors'].std()
    tissue_data['HH_contacts_total'] = HC.groupby(by=tissue_grp)['num_hair_neighbors'].sum()

    tissue_data['SH_contacts'] = SC.groupby(by=tissue_grp)['num_hair_neighbors'].mean()
    tissue_data['SH_contacts__std'] = SC.groupby(by=tissue_grp)['num_hair_neighbors'].std()

    tissue_data['num_HH_bonds'] = np.asarray(bonds.groupby(by=tissue_grp).apply(lambda bonds: (bonds['type'] == 'HH').sum()), dtype=int)
    tissue_data['num_HS_bonds'] = np.asarray(bonds.groupby(by=tissue_grp).apply(lambda bonds: (bonds['type'] == 'HS').sum()), dtype=int)
    tissue_data['num_SS_bonds'] = np.asarray(bonds.groupby(by=tissue_grp).apply(lambda bonds: (bonds['type'] == 'SS').sum()), dtype=int)
    tissue_data['num_HC_bonds'] = tissue_data['num_HH_bonds'] + tissue_data['num_HS_bonds']
    tissue_data['num_bonds'] = tissue_data['num_HH_bonds'] + tissue_data['num_HS_bonds'] + tissue_data['num_SS_bonds']
    tissue_data['fraction_HH_bonds'] = tissue_data['num_HH_bonds'] / tissue_data['num_bonds']


    # spatial organisation
    tissue_data['hexatic_order'] = HC.groupby(by=tissue_grp)['hexatic_order'].mean()
    tissue_data['hexatic_order__std'] = HC.groupby(by=tissue_grp)['hexatic_order'].std()
    tissue_data['hexatic_order_corrected'] = HC.groupby(by=tissue_grp)['hexatic_order_corrected'].mean()
    tissue_data['hexatic_order_corrected__std'] = HC.groupby(by=tissue_grp)['hexatic_order_corrected'].std()
    tissue_data['hexatic_order_ellipse_fit'] = HC.groupby(by=tissue_grp)['hexatic_order_ellipse_fit'].mean()
    tissue_data['hexatic_order_ellipse_fit__std'] = HC.groupby(by=tissue_grp)['hexatic_order_ellipse_fit'].std()

    tissue_data['num_next_HC_neighbors'] = HC.groupby(by=tissue_grp)['num_next_HC_neighbors'].mean()
    tissue_data['num_next_HC_neighbors__std'] = HC.groupby(by=tissue_grp)['num_next_HC_neighbors'].std()


    # polarity
    tissue_data['num_cilia'] = cells.groupby(by=tissue_grp)['cilium_DX'].count()

    tissue_data['polarity'] = HC.groupby(by=tissue_grp)[['cilium_DX', 'cilium_DY']].apply(circular_mean)
    tissue_data['polarity_angles'] = tissue_data['polarity'] / math.pi * 180.
    tissue_data['polarity__variance'] = HC.groupby(by=tissue_grp)[['cilium_DX', 'cilium_DY']].apply(circular_variance)

    HC['cilium_DX_corrected'] = HC['cilium_rho_corrected'] * np.cos(HC['cilium_phi_corrected'])
    HC['cilium_DY_corrected'] = HC['cilium_rho_corrected'] * np.sin(HC['cilium_phi_corrected'])
    tissue_data['polarity_corrected'] = HC.groupby(by=tissue_grp)[['cilium_DX_corrected', 'cilium_DY_corrected']].apply(circular_mean)
    tissue_data['polarity_corrected_angles'] = tissue_data['polarity_corrected'] / math.pi * 180.
    tissue_data['polarity_corrected__stddev'] = HC.groupby(by=tissue_grp)[['cilium_DX_corrected', 'cilium_DY_corrected']].apply(circular_stddev)
    tissue_data['polarity_corrected__stddev_angles'] = tissue_data['polarity_corrected__stddev'] / math.pi * 180
    tissue_data['polarity_corrected__variance'] = HC.groupby(by=tissue_grp)[['cilium_DX_corrected', 'cilium_DY_corrected']].apply(circular_variance)

    if 'cilium_DX_corrected__aligned' in tissue_data.columns:
        tissue_data['polarity_corrected_aligned'] = HC.groupby(by=tissue_grp)[['cilium_DX_corrected__aligned', 'cilium_DY_corrected__aligned']].apply(circular_mean)
        tissue_data['polarity_corrected_aligned_angles'] = tissue_data['polarity_corrected_aligned'] / 180 * math.pi
        tissue_data['polarity_corrected_aligned__stddev'] = HC.groupby(by=tissue_grp)[['cilium_DX_corrected__aligned', 'cilium_DY_corrected__aligned']].apply(circular_stddev)
        tissue_data['polarity_corrected_aligned__stddev_angles'] = np.where(
            tissue_data['num_cilia'] >= 10,
            tissue_data['polarity_corrected_aligned__stddev'] / math.pi * 180,
            np.nan
        )
        tissue_data['polarity_corrected_aligned__variance'] = HC.groupby(by=tissue_grp)[['cilium_DX_corrected__aligned', 'cilium_DY_corrected__aligned']].apply(circular_variance)

    mask = tissue_data['num_cilia'] < 10
    tissue_data.loc[mask, 'polarity'] = np.nan
    tissue_data.loc[mask, 'polarity__variance'] = np.nan
    tissue_data.loc[mask, 'polarity_corrected'] = np.nan
    tissue_data.loc[mask, 'polarity_corrected__variance'] = np.nan
    tissue_data.loc[mask, 'polarity_corrected__stddev'] = np.nan


    return tissue_data.reset_index()

In [19]:
__tissue_grp = [
    'Experiment',
    'Experiment_Series',
    'Treatment',
    'is_Control',
    'aligned_with_PD_axis',
    'stage',
    'Stage',
    'position',
    'SI_position',
    'S-I position',
    'SI_position_long',
    'PD_position',
    'P-D position',
]

tissue_grp = [
    'file_id',
    'sample_id',
    'filename',
    'filepath',
]

for g in __tissue_grp:
    tissue_grp.append(g)

In [20]:
tissue_data = tissue_averages(cells, bonds, tissue_grp)

/home/julian/image_analysis/BP_segmentation/utils.py:144: RuntimeWarning: invalid value encountered in scalar divide
  mean_x = x.sum() / norm
/home/julian/image_analysis/BP_segmentation/utils.py:145: RuntimeWarning: invalid value encountered in scalar divide
  mean_y = y.sum() / norm
/home/julian/image_analysis/BP_segmentation/utils.py:144: RuntimeWarning: invalid value encountered in scalar divide
  mean_x = x.sum() / norm
/home/julian/image_analysis/BP_segmentation/utils.py:145: RuntimeWarning: invalid value encountered in scalar divide
  mean_y = y.sum() / norm
/home/julian/image_analysis/BP_segmentation/utils.py:144: RuntimeWarning: invalid value encountered in scalar divide
  mean_x = x.sum() / norm
/home/julian/image_analysis/BP_segmentation/utils.py:145: RuntimeWarning: invalid value encountered in scalar divide
  mean_y = y.sum() / norm


### Summary and export

In [21]:
cells_experiments = cells.copy()
bonds_experiments = bonds.copy()

cells = cells.loc[cells['Experiment'] == 'Control']
bonds = bonds.loc[bonds['Experiment'] == 'Control']

# select subsets
HC = cells.loc[cells['is_HC']].copy()
SC = cells.loc[~cells['is_HC']].copy()

tissue_data__experiments = tissue_data.copy()
tissue_data = tissue_data.loc[tissue_data['Experiment'] == 'Control']

In [22]:
out_path = "{}/{}/{}".format(basepath, data_basedir, save_file_prefix)
cells.to_csv("{}_cells.csv".format(out_path))
bonds.to_csv("{}_bonds.csv".format(out_path))

cells_experiments.to_csv("{}_cells_w_treatments.csv".format(out_path))
bonds_experiments.to_csv("{}_bonds_w_treatments.csv".format(out_path))

tissue_data.to_csv("{}_tissue_averages.csv".format(out_path))
tissue_data__experiments.to_csv("{}_tissue_averages_w_treatments.csv".format(out_path))

In [23]:
print("Number of data-points")

grp = ['stage', 'position']

data_information = pd.DataFrame()
data_information['cells'] = cells.groupby(by=grp)[['num_neighbors']].count()
data_information['HC (cilia)'] = HC.groupby(by=['stage', 'position']).apply(
    lambda hc: (hc['num_neighbors'].count(),
                hc['cilium_DX'].count())
)
data_information['SC'] = SC.groupby(by=grp)[['num_neighbors']].count()
data_information['tissues (w. cilia)'] = HC.groupby(by=grp).apply(
    lambda hc: (len(np.unique(hc['file_id'])),
                len(np.unique(hc.loc[~np.isnan(hc['cilium_DX']), 'file_id'])))
)

print(data_information.to_string())
data_information.to_csv("{}_cells_infos.csv".format(out_path))

data_information = None
del data_information

Number of data-points
                cells  HC (cilia)    SC tissues (w. cilia)
stage position                                            
E10   0           941    (176, 0)   765             (1, 0)
      100        1127    (308, 0)   819             (1, 0)
      25I        2604  (601, 225)  2003             (4, 2)
      25S        1498    (408, 0)  1090             (2, 0)
      50I        2219  (621, 434)  1598             (2, 2)
      50S         704    (226, 0)   478             (1, 0)
      75I        3015  (824, 363)  2191             (7, 3)
      75S         804    (209, 0)   595             (1, 0)
E12   0           221    (45, 43)   176             (1, 1)
      100         163    (55, 51)   108             (1, 1)
      25I        2878  (623, 521)  2255             (5, 4)
      25S         314    (81, 62)   233             (1, 1)
      50I         236    (57, 57)   179             (1, 1)
      75I        1132  (319, 255)   813             (4, 3)
E14   100         887  (329, 126) 

In [24]:
print("\nNumber of data-points with known P-D axis")
aligned_cells = cells.loc[cells['aligned_with_PD_axis']].copy()

data_information = pd.DataFrame()
data_information['cells'] = aligned_cells.groupby(by=grp)[['num_neighbors']].count()
data_information['HC (cilia)'] = aligned_cells.loc[aligned_cells['is_HC']].groupby(by=grp).apply(
    lambda hc: (hc['num_neighbors'].count(), hc['cilium_DX'].count()))
data_information['SC'] = aligned_cells.loc[~aligned_cells['is_HC']].groupby(by=grp)[['num_neighbors']].count()
data_information['tissues (w. cilia)'] = aligned_cells.loc[aligned_cells['is_HC']].groupby(by=grp).apply(
    lambda hc: (len(np.unique(hc['file_id'])), len(np.unique(hc.loc[~np.isnan(hc['cilium_DX']), 'file_id']))))

print(data_information.to_string())
data_information.to_csv("{}_cells_aligned_infos.csv".format(out_path))

aligned_cells = None
data_information = None
del aligned_cells, data_information


Number of data-points with known P-D axis
                cells  HC (cilia)    SC tissues (w. cilia)
stage position                                            
E10   25I         996  (227, 225)   769             (2, 2)
      50I        1622  (442, 434)  1180             (2, 2)
      75I        1470  (363, 363)  1107             (3, 3)
E12   25I        2042  (396, 370)  1646             (3, 3)
      50I         236    (57, 57)   179             (1, 1)
      75I         935  (262, 255)   673             (3, 3)
E14   100         887  (329, 126)   558             (1, 1)
      25I         979  (180, 117)   799             (2, 2)
      25S         724   (148, 89)   576             (2, 2)
      50I         635  (153, 108)   482             (1, 1)
      50S         335    (103, 0)   232             (1, 0)
      75I        1656   (399, 91)  1257             (3, 2)
      75S        1362  (386, 206)   976             (2, 2)
E8    25I        1317  (332, 328)   985             (3, 3)
      50I    

In [25]:
print("\nNumber of data-points treatments")

grp = ['Experiment', 'stage', 'position']

data_information = pd.DataFrame()

data_information['cells'] = cells_experiments.groupby(by=grp)[['num_neighbors']].count()
data_information['HC (cilia)'] = cells_experiments.loc[cells_experiments['is_HC']].groupby(by=grp).apply(
    lambda hc: (hc['num_neighbors'].count(), hc['cilium_DX'].count()))
data_information['SC'] = cells_experiments.loc[~cells_experiments['is_HC']].groupby(by=grp)[['num_neighbors']].count()
data_information['tissues (w. cilia)'] = cells_experiments.loc[cells_experiments['is_HC']].groupby(by=grp).apply(
    lambda hc: (len(np.unique(hc['file_id'])), len(np.unique(hc.loc[~np.isnan(hc['cilium_DX']), 'file_id']))))

print(data_information.to_string())
data_information.to_csv("{}_cells_w_treatments_infos.csv".format(out_path))

data_information = None
del data_information


Number of data-points treatments
                                  cells  HC (cilia)    SC tissues (w. cilia)
Experiment        stage position                                            
Atoh1KO           E10   25I        2401  (342, 185)  2059             (4, 4)
Atoh1control      E10   25I        1354  (359, 203)   995             (4, 4)
Control           E10   0           941    (176, 0)   765             (1, 0)
                        100        1127    (308, 0)   819             (1, 0)
                        25I        2604  (601, 225)  2003             (4, 2)
                        25S        1498    (408, 0)  1090             (2, 0)
                        50I        2219  (621, 434)  1598             (2, 2)
                        50S         704    (226, 0)   478             (1, 0)
                        75I        3015  (824, 363)  2191             (7, 3)
                        75S         804    (209, 0)   595             (1, 0)
                  E12   0           221   

In [26]:
data = pd.DataFrame()
data['Series'] = tissue_data__experiments['Experiment_Series']
data['Experiment'] = tissue_data__experiments['Experiment']
data['Experiment'] = tissue_data__experiments['Experiment']
data['Name of the experiment'] = tissue_data__experiments.apply(lambda data: f"{data['stage']} {data['position']}" if data['Experiment_Series']=='Control' else data['Experiment'], axis=1)
data['Number of HCs (cilia) / SCs examined'] = tissue_data__experiments.apply(lambda data: f"{data['num_HC']} ({data['num_cilia']}) / {data['num_SC']}", axis=1)
data['Surface area of HCs'] = tissue_data__experiments['HC_area__no_norm']
data['Surface area of SCs'] = tissue_data__experiments['SC_area__no_norm']
data['Average HC neighbour number'] = tissue_data__experiments['coordination_number']
data['Corrected HC neighbour number'] = tissue_data__experiments['coordination_number__corr']
data['Average hexatic order (corrected)'] = tissue_data__experiments['hexatic_order_corrected']
data['Mean orientation'] = tissue_data__experiments['polarity_corrected_angles']
data['CSD'] = tissue_data__experiments['polarity_corrected__stddev_angles']

def rename_experiment(name):
    if name == 'Vangl2Control':
        return 'Vangl2 Control'
    if name == 'Vangl2KO':
        return 'Vangl2 KO'
    if name == 'Atoh1control':
        return 'Atoh1 Control'
    if name == 'Atoh1KO':
        return 'Atoh1 KO'
    if name == 'MLCKdmso-4hrs':
        return 'DMSO 4 hrs'
    if name == 'MLCKdmso-16hrs':
        return 'DMSO 16 hrs'
    if name == 'MLCKml7-4hrs':
        return 'ML7 4 hrs'
    if name == 'MLCKml7-16hrs':
        return 'ML7 4 + 12 hrs'
    if name == 'ROCKcontrol-4hrs':
        return 'DMEM 4 hrs'
    if name == 'ROCKcontrol-16hrs':
        return 'DMEM 16 hrs'
    if name == 'ROCKy27632-4hrs':
        return 'Y-27632 4 hrs'
    if name == 'ROCKy27632-16hrs':
        return 'Y-27632 4 + 12 hrs'
    if name == 'PTXcontrol':
        return 'DMEM 4 hrs (PTX control)'
    if name == 'PTXinhibition':
        return 'PTX 4 hrs'
    else:
        return name
data['Name of the experiment'] = data['Name of the experiment'].apply(rename_experiment)

data.loc[data['Series']=='Control'].to_csv(f"{basepath}/{data_basedir}/{save_file_prefix}_tissue_averages_publication_BP.csv")
data.loc[(data['Series']=='Control') & ((tissue_data__experiments['position']=='25I') | (tissue_data__experiments['position']=='75I'))].to_csv(f"{basepath}/{data_basedir}/{save_file_prefix}_tissue_averages_publication_BP_25I_75I.csv")
data.loc[data['Series']!='Control'].to_csv(f"{basepath}/{data_basedir}/{save_file_prefix}_tissue_averages_publication_treatments.csv")

In [27]:
print("WARNING tissues with more than 5% H-H contacts (false contacts)")
print(tissue_data__experiments.sort_values(by='fraction_HH_bonds', ascending=False)[['filename', 'file_id', 'is_Control', 'aligned_with_PD_axis', 'fraction_HH_bonds', 'num_HH_bonds']].to_string())

WARNING tissues with more than 5% H-H contacts (false contacts)
                    filename  file_id  is_Control  aligned_with_PD_axis  fraction_HH_bonds  num_HH_bonds
102    ROCKy27632-4hrs_3.tif       97       False                  True           0.032117            22
98    ROCKy27632-16hrs_2.tif       93       False                  True           0.027907            12
101    ROCKy27632-4hrs_2.tif       96       False                  True           0.026919            27
87        MLCKml7-4hrs_3.tif       82       False                  True           0.020478             6
100    ROCKy27632-4hrs_1.tif       95       False                  True           0.019493            10
85        MLCKml7-4hrs_1.tif       80       False                  True           0.013183             7
66        E10_Vangl2KO_3.tif       61       False                 False           0.011396             4
86        MLCKml7-4hrs_2.tif       81       False                  True           0.010256      

# Schematics

In [28]:
fig, ax = plt.subplots(1, 1, figsize=[3, 3])

phi = np.linspace(0, 2 * math.pi, 101)

long = 3
short = 1
rotate = -30/180*math.pi - math.pi
angle = 15

def __rescale_dx_dy(displ_x, displ_y, *, long, short, theta, scatter=False):
    """Rescale to circle for correction"""
    dx_ = (displ_x * np.cos(-theta) - displ_y * np.sin(-theta)) / (long / short)
    dy_ =  displ_x * np.sin(-theta) + displ_y * np.cos(-theta)

    dx = dx_ * np.cos(theta) - dy_ * np.sin(theta)
    dy = dx_ * np.sin(theta) + dy_ * np.cos(theta)

    rho = (long * short)**0.5

    return dx/short, dy/short

def Rho_ellipse(angle, *, short, long, rotate):
    return long * short / (long**2 * np.sin(angle - rotate)**2 + short**2 * np.cos(angle - rotate)**2)**0.5

# the circle with equal area
area = math.pi * long * short
rho = (area / math.pi)**0.5
ax.plot(rho * np.cos(phi), rho * np.sin(phi), 
        color='black', linestyle='dashed', lw=0.5)

rho_ellipse = Rho_ellipse(phi, short=short, long=long, rotate=rotate)
ax.plot(rho_ellipse * np.cos(phi), rho_ellipse * np.sin(phi),
        color='black', lw=0.5)

cilium_angle = (np.array([10]))/180 * math.pi + rotate
cilium_rho = 0.85 * Rho_ellipse(cilium_angle, 
                                short=short, long=long, rotate=rotate)
ax.scatter(
    cilium_rho* np.cos(cilium_angle), cilium_rho * np.sin(cilium_angle), 
    color='red', s=20
)

dx, dy = __rescale_dx_dy(
    cilium_rho * np.cos(cilium_angle), cilium_rho * np.sin(cilium_angle), 
    long=long, short=short, theta=rotate, scatter=True)
ax.quiver(
    np.zeros_like(dx), np.zeros_like(dx), dx*rho, dy*rho, 
    scale=1, angles='xy', scale_units='xy', color='red'
)

__rho = Rho_ellipse(0, short=short, long=long, rotate=0)
x, y = __rho * np.cos(rotate), __rho * np.sin(rotate)
ax.quiver(
    x, y, -0.9 * x / __rho * (long - rho), -0.9 * y / __rho * (long - rho), 
    scale=1, angles='xy', scale_units='xy', pivot='tip', color='gray'
)
ax.quiver(
    -x, -y, 0.9 * x / __rho * (long - rho), 0.9 * y / __rho * (long - rho), 
    scale=1, angles='xy', scale_units='xy', pivot='tip', color='gray'
)

__rho = Rho_ellipse(90, short=short, long=long, rotate=0)
x, y = -__rho * np.sin(rotate), __rho * np.cos(rotate)
ax.quiver(
    x, y, 0.9 * x / __rho * (long - rho), 0.9 * y / __rho * (long - rho), 
    scale=1, angles='xy', scale_units='xy', color='gray'
)
ax.quiver(
    -x, -y, -0.9 * x / __rho * (long - rho), -0.9 * y / __rho * (long - rho), 
    scale=1, angles='xy', scale_units='xy', color='gray'
)

ax.set_xlim([-4, 4])
ax.set_ylim([-3, 3])
ax.set_aspect('equal')


ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)

ax.quiver(
    0, 0, 2.5*long, 0, 
    scale=1, angles='xy', scale_units='xy', color='black', 
    width=0.0025, pivot='mid'
)
ax.quiver(
    0, 0, 0, long*1.75, 
    scale=1, angles='xy', scale_units='xy', color='black', 
    width=0.0025, pivot='mid'
)

plt.savefig(f"{figure_path}/cell_elongation_correction.svg",format='svg')

In [29]:
fig, ax = plt.subplots(1, 1, figsize=[3, 3])

# the circle with equal area
area = math.pi * long * short
rho = (area / math.pi)**0.5
ax.plot(rho * np.cos(phi), rho * np.sin(phi), 
        color='black', linestyle='dashed', lw=0.5)

rho_ellipse = Rho_ellipse(phi, short=short, long=long, rotate=rotate)
ax.plot(rho_ellipse * np.cos(phi), rho_ellipse * np.sin(phi), 
        color='black', lw=0.5)

# cilium_angle = (np.linspace(0, 90, 4))/180 * math.pi + rotate
cilium_angle = (np.linspace(0, 360, 6, endpoint=False)+15)/180*math.pi + rotate
cilium_rho = Rho_ellipse(cilium_angle, short=short, long=long, rotate=rotate)
_dx, _dy = __rescale_dx_dy(
    cilium_rho * np.cos(cilium_angle), cilium_rho * np.sin(cilium_angle), 
    long=short, short=long, theta=rotate, scatter=True
)
cilium_angle = np.arctan2(_dy, _dx)[2:5]
cilium_rho = 2. * Rho_ellipse(cilium_angle, 
                              short=short, long=long, rotate=rotate)

dx, dy = __rescale_dx_dy(
    cilium_rho * np.cos(cilium_angle), cilium_rho * np.sin(cilium_angle),
    long=long, short=short, theta=rotate, scatter=True
)
print(np.arctan2(dy, dx)/math.pi * 180)
ax.scatter(
    cilium_rho/2* np.cos(cilium_angle), cilium_rho/2 * np.sin(cilium_angle), 
    color='blue', s=20
)
ax.quiver(
    np.zeros_like(dx), np.zeros_like(dx), 
    cilium_rho* np.cos(cilium_angle), cilium_rho * np.sin(cilium_angle), 
    scale=1, angles='xy', scale_units='xy', color='blue'
)
ax.scatter(dx*rho/2, dy*rho/2, color='red')
ax.quiver(
    np.zeros_like(dx), np.zeros_like(dx), dx*rho, dy*rho, 
    scale=1, angles='xy', scale_units='xy', color='red'
)

__rho = Rho_ellipse(0, short=short, long=long, rotate=0)
x, y = __rho * np.cos(rotate), __rho * np.sin(rotate)
ax.quiver(
    x, y, -0.9 * x / __rho * (long - rho), -0.9 * y / __rho * (long - rho), 
    scale=1, angles='xy', scale_units='xy', pivot='tip', color='gray'
)
ax.quiver(
    -x, -y, 0.9 * x / __rho * (long - rho), 0.9 * y / __rho * (long - rho), 
    scale=1, angles='xy', scale_units='xy', pivot='tip', color='gray'
)

__rho = Rho_ellipse(90, short=short, long=long, rotate=0)
x, y = -__rho * np.sin(rotate), __rho * np.cos(rotate)
ax.quiver(
    x, y, 0.9 * x / __rho * (long - rho), 0.9 * y / __rho * (long - rho), 
    scale=1, angles='xy', scale_units='xy', color='gray'
)
ax.quiver(
    -x, -y, -0.9 * x / __rho * (long - rho), -0.9 * y / __rho * (long - rho), 
    scale=1, angles='xy', scale_units='xy', color='gray'
)

ax.set_xlim([-4, 4])
ax.set_ylim([-3, 3])
ax.set_aspect('equal')


ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)
# ax.axis('off')

ax.quiver(
    0, 0, 2.5*long, 0, 
    scale=1, angles='xy', scale_units='xy', color='black', 
    width=0.0025, pivot='mid'
)
ax.quiver(
    0, 0, 0, long*1.75, 
    scale=1, angles='xy', scale_units='xy', color='black', 
    width=0.0025, pivot='mid'
)

plt.savefig(f"{figure_path}/cell_hexatic_correction.svg",format='svg')

[-75. -15.  45.]


# Analysis

## Area

In [30]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=[5.9/2, 5.9/3])

points = sns.pointplot(
    data=cells.loc[cells['position'] == '25I'],
    ax=ax1,
    x='stage',
    y='area',
    hue='is_HC',
    hue_order=[True, False],
    palette=['gray', 'red'],
    join=False,
    ci='sd',
    # size='stage',
    # sizes=[2],
    # capsize=.05,
    dodge=True,
    errwidth=linewidth
)
plt.setp(points.collections, sizes=[markersize])


ax1.set_ylim([0, None])
ax1.set_xlim([-0.5, 3.5])

ax1.set_ylabel('Cell area [$\mathrm{\mu m^2}$]')
ax1.set_ylim(0, 70)
ax1.set_yticks([0, 20, 40, 60])

ax1.set_xlabel('')
ax1.set_xticklabels(['E8', 'E10', 'E12', 'E14'], fontsize=labelsize)

leg_handles = ax1.get_legend_handles_labels()[0]
ax1.legend(leg_handles, ['HC', 'SC'], title='')



points = sns.pointplot(
    data=cells.loc[cells['is_HC']],
    ax=ax2,
    x='stage',
    y='normalized_area_cells',
    hue='position',
    hue_order=['25I', '75I'],
    palette='Oranges',
    join=False,
    ci='sd',
    # capsize=.05,
    dodge=True,
    errwidth=linewidth
)
plt.setp(points.collections, sizes=[markersize])


leg_handles = ax2.get_legend_handles_labels()[0]
ax2.legend(leg_handles, ['25I', '75I'], title='')


ax2.hlines(y=[1], xmin=-1, xmax=5, ls=':', color='gray')

ax2.set_ylim([0, 5])
ax2.set_xlim([-0.5, 3.5])

ax2.set_ylabel('Normalised HC area')

ax2.set_xlabel('')
ax2.set_xticklabels(['E8', 'E10', 'E12', 'E14'], fontsize=labelsize)

plt.savefig(f"{figure_path}/area_development__explants.svg",format='svg')

/tmp/ipykernel_79459/1281265678.py:3: FutureWarning: 

The `ci` parameter is deprecated. Use `errorbar='sd'` for the same effect.

  points = sns.pointplot(
/tmp/ipykernel_79459/1281265678.py:37: FutureWarning: 

The `ci` parameter is deprecated. Use `errorbar='sd'` for the same effect.

  points = sns.pointplot(
findfont: Font family ['cmtt10'] not found. Falling back to DejaVu Sans.
findfont: Font family ['cmb10'] not found. Falling back to DejaVu Sans.
findfont: Font family ['cmss10'] not found. Falling back to DejaVu Sans.
findfont: Font family ['DejaVu Sans Display'] not found. Falling back to DejaVu Sans.


In [31]:
fig, ax = plt.subplots(1, 1, figsize=[5.9/4, 5.9/3])

points = sns.pointplot(
    data=cells.loc[cells['is_HC']],
    x='stage',
    y='normalized_area_cells',
    hue='position', hue_order=['25I', '75I'],
    # palette=[''],
    join=False,
    errorbar='sd',
    capsize=.05,
    dodge=True,
    errwidth=linewidth,
    markers='.',
)
plt.setp(points.collections, sizes=[markersize])

print(HC.loc[HC['position'] == '25I'].groupby(by='stage')['normalized_area_cells'].mean())

ax.hlines(y=[1], xmin=-1, xmax=3.5, ls=':', color='gray')


leg_handles = ax.get_legend_handles_labels()[0]
ax.legend(leg_handles, ['25I', '75I'], title='')


ax.set_ylim([0, 4.5])
ax.set_xlim([-0.5, 3.5])

ax.set_ylabel('Normalised HC area')
ax.set_xlabel('')
ax.set_xticklabels(['E8', 'E10', 'E12', 'E14'], fontsize=labelsize)


plt.savefig(f"{figure_path}/HC_area.svg",format='svg')

stage
E10    0.881518
E12    2.056125
E14    4.059736
E8     0.637964
Name: normalized_area_cells, dtype: float64


In [32]:
fig, ax = plt.subplots(1, 1, figsize=[5.9/4, 5.9/3])

points = sns.pointplot(
    data=cells.loc[cells['position'] == '25I'],
    x='stage',
    y='area',
    hue='is_HC',
    hue_order=[False, True],
    palette=['gray', 'red'],
    join=False,
    errorbar='sd',
    capsize=.05,
    dodge=True,
    errwidth=linewidth,
    markers='.',
)
plt.setp(points.collections, sizes=[markersize])

print(HC.loc[HC['position'] == '25I'].groupby(by='stage')['area'].mean())


ax.set_ylim([0, None])
ax.set_xlim([-0.5, 3.5])

ax.set_ylabel('Cell area [$\mu m^2$]')

ax.set_xlabel('')
ax.set_xticklabels(['E8', 'E10', 'E12', 'E14'], fontsize=labelsize)

leg_handles = ax.get_legend_handles_labels()[0]
ax.legend(leg_handles, ['SC', 'HC'], title='')

plt.savefig(f"{figure_path}/HC_area__no_norm.svg",format='svg')

stage
E10     7.259460
E12    24.270634
E14    69.356152
E8      5.868119
Name: area, dtype: float64


In [33]:
fig, ax = plt.subplots(1, 1, figsize=[5.9/4, 5.9/3])
fig.tight_layout()

sns.pointplot(
    data=tissue_data.loc[tissue_data['position']=='25I'],
    x='stage',
    y='SC_area__no_norm',
    palette='Oranges',
    color='gray',
    join=False,
    errorbar='sd',
    capsize=.05,
    errwidth=linewidth,
    dodge=True,
    markers='.',
)

sns.pointplot(
    data=tissue_data.loc[tissue_data['position']=='25I'],
    x='stage',
    y='HC_area__no_norm',
    color='red',
    join=False,
    errorbar='sd',
    capsize=.05,
    errwidth=linewidth,
    dodge=True,
    markers='.',
)

ax.set_ylim([0, None])
ax.set_xlim([-0.5, 3.5])
ax.set_xticklabels([8, 10, 12, 14])

ax.set_ylabel('Cell area [$\mu m^2 $]')
ax.set_xlabel('')
ax.set_xticklabels(['E8', 'E10', 'E12', 'E14'], fontsize=labelsize)

# Mean of mean
plt.savefig(f"{figure_path}/HC_area__no_norm__tissues.svg",format='svg')

In [34]:
fig, ax = plt.subplots(1, 1, figsize=[5.9/4, 5.9/3])

points = sns.pointplot(
    data=HC,
    x='stage',
    y='area',
    hue='position', hue_order=['25I', '75I'],
    join=False,
    errorbar='sd',
    capsize=.05,
    dodge=True,
    errwidth=linewidth,
    markers='.',
)
plt.setp(points.collections, sizes=[markersize])

ax.legend().remove()

box_pairs=[
    ('E8', 'E10'),
    ('E8', 'E12'),
    ('E8', 'E14'),
]

annotator = Annotator(
    ax,
    box_pairs,
    data=HC,
    x='stage',
    y='area',
    split=True,
)
annotator.configure(
    test='Mann-Whitney',
    line_width=0.75,
)

ax.set_ylim([0, None])

ax.set_ylabel('HC area [$\mu m^2$]')
ax.set_xlabel('')
ax.set_xticklabels(['E8', 'E10', 'E12', 'E14'], fontsize=labelsize)

annotator.apply_and_annotate()


p-value annotation legend:
      ns: p <= 1.00e+00
       *: 1.00e-02 < p <= 5.00e-02
      **: 1.00e-03 < p <= 1.00e-02
     ***: 1.00e-04 < p <= 1.00e-03
    ****: p <= 1.00e-04

E8 vs. E10: Mann-Whitney-Wilcoxon test two-sided, P_val:3.077e-130 U_stat=1.470e+06
E8 vs. E12: Mann-Whitney-Wilcoxon test two-sided, P_val:0.000e+00 U_stat=2.638e+04
E8 vs. E14: Mann-Whitney-Wilcoxon test two-sided, P_val:0.000e+00 U_stat=7.157e+04


/home/julian/.local/lib/python3.10/site-packages/statannotations/_Plotter.py:337: UserWarning: Invalid x-position found. Are the same parameters passed to seaborn and statannotations calls? or are there few data points?
  warnings.warn(
/home/julian/.local/lib/python3.10/site-packages/statannotations/_Plotter.py:337: UserWarning: Invalid x-position found. Are the same parameters passed to seaborn and statannotations calls? or are there few data points?
  warnings.warn(


(<AxesSubplot:ylabel='HC area [$\\mu m^2$]'>,
  <statannotations.Annotation.Annotation at 0x7f57f1eec250>])

In [35]:
fig, ax = plt.subplots(1, 1, figsize=[5.9/4, 5.9/3])

points = sns.pointplot(
    data=SC,
    x='stage',
    y='area',
    hue='position', hue_order=['25I', '75I'],
    join=False,
    errorbar='sd',
    capsize=.05,
    dodge=True,
    errwidth=linewidth,
    markers='.',
)
plt.setp(points.collections, sizes=[markersize])

ax.legend().remove()

box_pairs=[
    ('E8', 'E10'),
    ('E8', 'E12'),
    ('E8', 'E14'),
]

annotator = Annotator(
    ax,
    box_pairs,
    data=SC,
    x='stage',
    y='area',
    split=True,
)
annotator.configure(
    test='Mann-Whitney',
    line_width=0.75,
)


ax.set_ylim([0, None])

ax.set_ylabel('Cell area [$\mu m^2$]')
ax.set_xlabel('')
ax.set_xticklabels(['E8', 'E10', 'E12', 'E14'], fontsize=labelsize)

annotator.apply_and_annotate()


p-value annotation legend:
      ns: p <= 1.00e+00
       *: 1.00e-02 < p <= 5.00e-02
      **: 1.00e-03 < p <= 1.00e-02
     ***: 1.00e-04 < p <= 1.00e-03
    ****: p <= 1.00e-04

E8 vs. E10: Mann-Whitney-Wilcoxon test two-sided, P_val:8.616e-01 U_stat=2.713e+07
E8 vs. E12: Mann-Whitney-Wilcoxon test two-sided, P_val:1.644e-01 U_stat=1.087e+07
E8 vs. E14: Mann-Whitney-Wilcoxon test two-sided, P_val:5.604e-03 U_stat=1.342e+07


/home/julian/.local/lib/python3.10/site-packages/statannotations/_Plotter.py:337: UserWarning: Invalid x-position found. Are the same parameters passed to seaborn and statannotations calls? or are there few data points?
  warnings.warn(
/home/julian/.local/lib/python3.10/site-packages/statannotations/_Plotter.py:337: UserWarning: Invalid x-position found. Are the same parameters passed to seaborn and statannotations calls? or are there few data points?
  warnings.warn(


(<AxesSubplot:ylabel='Cell area [$\\mu m^2$]'>,
  <statannotations.Annotation.Annotation at 0x7f57e38376a0>])

In [36]:
area_increase = pd.DataFrame()

area_increase['HC_area__E8'] = HC.loc[HC['stage']=='E8'].groupby(by='position')['normalized_area_cells'].mean()
area_increase['HC_area__E10'] = HC.loc[HC['stage']=='E10'].groupby(by='position')['normalized_area_cells'].mean()
area_increase['HC_area__E12'] = HC.loc[HC['stage']=='E12'].groupby(by='position')['normalized_area_cells'].mean()
area_increase['HC_area__E14'] = HC.loc[HC['stage']=='E14'].groupby(by='position')['normalized_area_cells'].mean()

area_increase['SC_area__E8'] = SC.loc[SC['stage']=='E8'].groupby(by='position')['normalized_area_cells'].mean()
area_increase['SC_area__E10'] = SC.loc[SC['stage']=='E10'].groupby(by='position')['normalized_area_cells'].mean()
area_increase['SC_area__E12'] = SC.loc[SC['stage']=='E12'].groupby(by='position')['normalized_area_cells'].mean()
area_increase['SC_area__E14'] = SC.loc[SC['stage']=='E14'].groupby(by='position')['normalized_area_cells'].mean()

area_increase['ration__E14_E8'] = area_increase['HC_area__E14'] / area_increase['HC_area__E8']
area_increase['ration__E12_E10'] = area_increase['HC_area__E12'] / area_increase['HC_area__E10']
area_increase['ration__E12_E8'] = area_increase['HC_area__E12'] / area_increase['HC_area__E8']


area_increase['std__HC_area__E8'] = HC.loc[HC['stage']=='E8'].groupby(by='position')['normalized_area_cells'].std()
area_increase['std__HC_area__E10'] = HC.loc[HC['stage']=='E10'].groupby(by='position')['normalized_area_cells'].std()
area_increase['std__HC_area__E12'] = HC.loc[HC['stage']=='E12'].groupby(by='position')['normalized_area_cells'].std()
area_increase['std__HC_area__E14'] = HC.loc[HC['stage']=='E14'].groupby(by='position')['normalized_area_cells'].std()

area_increase['std__SC_area__E8'] = SC.loc[SC['stage']=='E8'].groupby(by='position')['normalized_area_cells'].std()
area_increase['std__SC_area__E10'] = SC.loc[SC['stage']=='E10'].groupby(by='position')['normalized_area_cells'].std()
area_increase['std__SC_area__E12'] = SC.loc[SC['stage']=='E12'].groupby(by='position')['normalized_area_cells'].std()
area_increase['std__SC_area__E14'] = SC.loc[SC['stage']=='E14'].groupby(by='position')['normalized_area_cells'].std()

print(area_increase.to_string())

print("No normalisation")
area_increase['HC_area__no_norm__E8'] = HC.loc[HC['stage']=='E8'].groupby(by='position')['area'].mean()
area_increase['HC_area__no_norm__E10'] = HC.loc[HC['stage']=='E10'].groupby(by='position')['area'].mean()
area_increase['HC_area__no_norm__E12'] = HC.loc[HC['stage']=='E12'].groupby(by='position')['area'].mean()
area_increase['HC_area__no_norm__E14'] = HC.loc[HC['stage']=='E14'].groupby(by='position')['area'].mean()
area_increase['ration__no_norm__E14_E8'] = area_increase['HC_area__no_norm__E14'] / area_increase['HC_area__no_norm__E8']
area_increase['ration__no_norm__E12_E10'] = area_increase['HC_area__no_norm__E12'] / area_increase['HC_area__no_norm__E10']
area_increase['ration__no_norm__E12_E8'] = area_increase['HC_area__no_norm__E12'] / area_increase['HC_area__no_norm__E8']

print(area_increase.to_string())

          HC_area__E8  HC_area__E10  HC_area__E12  HC_area__E14  SC_area__E8  SC_area__E10  SC_area__E12  SC_area__E14  ration__E14_E8  ration__E12_E10  ration__E12_E8  std__HC_area__E8  std__HC_area__E10  std__HC_area__E12  std__HC_area__E14  std__SC_area__E8  std__SC_area__E10  std__SC_area__E12  std__SC_area__E14
position                                                                                                                                                                                                                                                                                                                     
25I          0.637964      0.881518      2.056125      4.059736     1.087823      1.035551      0.708219      0.310698        6.363584         2.332483        3.222950          0.170299           0.246184           0.702092           0.600878          0.385378           0.339922           0.291579           0.133373
25S          0.650364      0.745388      2.269

## Neighbourhood

In [37]:
fig, ax = plt.subplots(1, 1, figsize=[5.9/5, 5.9/3])

points = sns.pointplot(
    data=cells.loc[cells['position'] == '25I'],
    x='stage',
    y='num_neighbors',
    hue='is_HC',
    hue_order=[False, True],
    palette=['gray', 'red'],
    join=False,
    errorbar='sd',
    capsize=.05,
    dodge=True,
    errwidth=linewidth,
    markers='.',
)
plt.setp(points.collections, sizes=[markersize])


ax.set_ylim([3, None])
ax.set_xlim([-0.5, 3.5])

ax.set_ylabel('Neighbour number')

ax.set_xlabel('')
ax.set_xticklabels(['E8', 'E10', 'E12', 'E14'], fontsize=labelsize)

leg_handles = ax.get_legend_handles_labels()[0]
ax.legend(leg_handles, ['SC', 'HC'], title='')

plt.savefig(f"{figure_path}/num_neighbors.svg",format='svg')

In [38]:
fig, ax = plt.subplots(1, 1, figsize=[5.9/4, 5.9/3])

points = sns.pointplot(
    data=cells.loc[cells['position']=='25I'],
    x='stage',
    y='num_neighbors',
    hue='is_HC',
    hue_order=[False, True],
    palette=['gray', 'red'],
    color='red',
    join=False,
    errorbar='sd',
    capsize=.05,
    dodge=True,
    # s=2,
    errwidth=linewidth,
    markers='.',
)
plt.setp(points.collections, sizes=[markersize])

ax.set_ylim([3, 11])
ax.yaxis.set_minor_locator(mpl.ticker.MultipleLocator(1))

ax.set_xlim([-0.5, 3.5])
ax.set_xlabel('')
ax.set_xticklabels(['E8', 'E10', 'E12', 'E14'], fontsize=labelsize)

ax.set_ylabel('Neighbour number')
# ax.set_yticks([0., 0.5, 1.])

ax.hlines(y=[6], xmin=-0.5, xmax=3.5, colors='black', lw=linewidth)

plt.savefig(f"{figure_path}/num_neighbors__25I.svg")

In [39]:
fig, ax = plt.subplots(1, 1, figsize=[5.9/5, 5.9/3])

points = sns.pointplot(
    data=HC,
    x='stage',
    y='num_neighbors',
    hue='position', hue_order=['25I', '75I'],
    join=False,
    errorbar='sd',
    capsize=.05,
    dodge=True,
    errwidth=linewidth,
    markers='.',
)
plt.setp(points.collections, sizes=[markersize])


leg_handles = ax.get_legend_handles_labels()[0]
ax.legend(leg_handles, ['25I', '75I'], title='')


ax.set_ylim([3, 11])
ax.set_xlabel('')
ax.set_xticklabels(['E8', 'E10', 'E12', 'E14'], fontsize=labelsize)


plt.savefig(f"{figure_path}/HC_neighbour_number.svg",format='svg')

In [40]:
fig, ax = plt.subplots(1, 1, figsize=[5.9/4, 5.9/3])

points = sns.pointplot(
    data=cells.loc[cells['position']=='25I'],
    x='stage',
    y='num_hair_neighbors',
    hue='is_HC',
    hue_order=[False, True],
    palette=['gray', 'red'],
    color='red',
    join=False,
    errorbar='sd',
    capsize=.05,
    dodge=True,
    # s=2,
    errwidth=linewidth,
    markers='.',
)
plt.setp(points.collections, sizes=[markersize])

ax.set_xlabel('')
ax.set_xticklabels(['E8', 'E10', 'E12', 'E14'], fontsize=labelsize)
ax.set_xlim([-0.5, 3.5])

ax.set_ylabel('HC neighbours')

ax.set_ylim([-.1, 2.5])
ax.set_yticks([0, 1, 2, 3])
ax.yaxis.set_minor_locator(mpl.ticker.MultipleLocator(0.5))

ax.hlines(y=[0], xmin=-0.5, xmax=3.5, colors='black', lw=linewidth, zorder=-np.inf)

plt.savefig(f"{figure_path}/hair_neighbour_numbers_25I.svg")

### Neighbourhood histogram plots

In [41]:
fig, ax = plt.subplots(1, 1, figsize=[5.9/4, 5.9/3])

mask = np.where(
    cells['HC_normalized_area']
)

data = cells.loc[(cells['stage'] == 'E10') & (cells['position'] == '25I')]
print(data['filename'].unique())

sns.histplot(
    data=data,
    x='num_neighbors',
    binrange=[3, 12],
    discrete=True,
    hue='is_HC',
    palette=['gray', 'red'],
    multiple='stack',
    stat='probability',
    common_norm=True,
    legend=False,
    alpha=0.5
)

__d = data
__hc = __d.loc[__d['is_HC']]
ax.vlines(
    x=[
        __d['num_neighbors'].mean(),
        __d.loc[__d['is_HC'], 'num_neighbors'].mean(),
        __d.loc[~__d['is_HC'], 'num_neighbors'].mean(),
    ],
    ymin=[
        0,
        0,
        __hc.loc[__hc['num_neighbors']== 6, 'num_neighbors'].count() / __d['num_neighbors'].count(),
    ],
    ymax=[
        __d.loc[__d['num_neighbors']== 6, 'num_neighbors'].count() / __d['num_neighbors'].count(),
        __hc.loc[__hc['num_neighbors']==5, 'num_neighbors'].count() / __d['num_neighbors'].count(),
        __d.loc[__d['num_neighbors']== 6, 'num_neighbors'].count() / __d['num_neighbors'].count(),
    ],
    colors=['black', 'red', 'gray'],
    lw=0.5,
    ls='dashed',
)

print([
        __d['num_neighbors'].mean(),
        __d.loc[__d['is_HC'], 'num_neighbors'].mean(),
        __d.loc[~__d['is_HC'], 'num_neighbors'].mean(),
    ])

ax.set_xlim([2.5, 10.5])
ax.set_xticks([4, 6, 8, 10])
ax.xaxis.set_minor_locator(mpl.ticker.AutoMinorLocator(2))

ax.set_ylim([0, 0.4])

ax.set_xlabel('Neighbour number')
ax.set_ylabel('Frequency')


plt.savefig(f"{figure_path}/neighbours_hc_vs_sc_stack.svg")

print(
    "normalised area:",
    "%.2f"%__d.loc[__d['is_HC'] == True, 'normalized_area_cells'].mean(),
    "pm %.2f"%__d.loc[__d['is_HC'] == True, 'normalized_area_cells'].std(),
    ". %.2f"%__d.loc[__d['is_HC'] == False, 'normalized_area_cells'].mean(),
    "pm %.2f"%__d.loc[__d['is_HC'] == False, 'normalized_area_cells'].std(),
)
print(
    "num neighbours:",
    "%.2f"%__d.loc[__d['is_HC'] == True, 'num_neighbors'].mean(),
    "pm %.2f"%__d.loc[__d['is_HC'] == True, 'num_neighbors'].std(),
    ". %.2f"%__d.loc[__d['is_HC'] == False, 'num_neighbors'].mean(),
    "pm %.2f"%__d.loc[__d['is_HC'] == False, 'num_neighbors'].std(),
)
print("Welch's t-test on neighbourhood:",
    "%.2E" % scipy.stats.ttest_ind(
        __d.loc[__d['is_HC'] == True, 'num_neighbors'],
        __d.loc[__d['is_HC'] == False, 'num_neighbors'],
        equal_var=False
    )[1]
)
print("Welch's t-test on area:",
    "%.2E" % scipy.stats.ttest_ind(
        __d.loc[__d['is_HC'] == True, 'normalized_area_cells'],
        __d.loc[__d['is_HC'] == False, 'normalized_area_cells'],
        equal_var=False
    )[1]
)

['E10_25I_1_0.tif' 'E10_25I_2_0.tif' 'E10_25I_1_1.tif' 'E10_25I_3_1.tif']
[5.943548387096774, 5.156405990016639, 6.17973040439341]
normalised area: 0.88 pm 0.25 . 1.04 pm 0.34
num neighbours: 5.16 pm 0.89 . 6.18 pm 0.99
Welch's t-test on neighbourhood: 1.64E-103
Welch's t-test on area: 1.03E-32


In [42]:
fig, ax = plt.subplots(1, 1, figsize=[5.9/4, 5.9/3])

mask = np.where(
    cells['HC_normalized_area']
)

data = cells.loc[(cells['stage'] == 'E10') & (cells['position'] == '25I')]

bins = [
    2.6, 3.4,
    3.6, 4.4,
    4.6, 5.4,
    5.6, 6.4,
    6.6, 7.4,
    7.6, 8.4,
    8.6, 9.4,
    9.6, 10.4,
    10.6
]

sns.histplot(
    data=data,
    x='num_neighbors',
    # binrange=[3, 12],
    bins=bins,
    # discrete=True,
    hue='is_HC',
    palette=['gray', 'red'],
    multiple='dodge',
    # multiple='stack',
    stat='probability',
    common_norm=False,
    legend=False,
    alpha=0.5
)

ax.set_xlim([2.5, 10.5])
ax.set_xticks([4, 6, 8, 10])
ax.xaxis.set_minor_locator(mpl.ticker.AutoMinorLocator(2))

# ax.set_ylim([0, 3050])
ax.set_ylim([0, 0.5])
ax.set_yticks([0., 0.2, 0.4])
ax.yaxis.set_minor_locator(mpl.ticker.AutoMinorLocator(2))

ax.set_xlabel('Neighbour number')
ax.set_ylabel('Frequency')


plt.savefig(f"{figure_path}/neighbours_hc_vs_sc_dodge.svg")


fig, ax = plt.subplots(1, 1, figsize=[5.9/4, 5.9/3])

sns.histplot(
    data=data,
    x='num_neighbors',
    binrange=[3, 12],
    # bins=bins,
    discrete=True,
    hue='is_HC',
    palette=['gray', 'red'],
    # multiple='dodge',
    # multiple='stack',
    stat='probability',
    common_norm=False,
    legend=False,
    alpha=0.5
)

ax.set_xlim([2.5, 10.5])
ax.set_xticks([4, 6, 8, 10])
ax.xaxis.set_minor_locator(mpl.ticker.AutoMinorLocator(2))

# ax.set_ylim([0, 3050])
ax.set_ylim([0, 0.5])
ax.set_yticks([0., 0.2, 0.4])
ax.yaxis.set_minor_locator(mpl.ticker.AutoMinorLocator(2))

ax.set_xlabel('Neighbour number')
ax.set_ylabel('Frequency')

print(data.loc[data['is_HC'], 'num_neighbors'].mean())
print(data.loc[~data['is_HC'], 'num_neighbors'].mean())


plt.savefig(f"{figure_path}/neighbours_hc_vs_sc_layer.svg")

5.156405990016639
6.17973040439341


In [43]:
def x_to_xtick(x, *, x_ticks):
    for i in range(len(x_ticks)):
        if x == x_ticks[i]:
            return 2 * i + 1
    raise

def histplot_violin(
    hist, *, 
    data_column, ax, label, 
    x_ticks, x_tick_labels, y_ticks, y_tick_labels, 
    set_colors, set_background=[],
    means=None,
    hist_split=None,
    means_split=None,
    split_color='green'
):
    ax.set_xlim([0, 2*len(x_ticks)])
    ax.set_xticks(np.arange(1, 2*len(x_ticks)+1, 2), x_tick_labels, rotation = 30)
    ax.tick_params(axis='x', which='major', pad=0)
    ax.xaxis.set_minor_locator(mpl.ticker.MultipleLocator(2))
    for label in ax.get_xmajorticklabels():
        label.set_horizontalalignment("right")
    
    ax.set_yticks(
        np.arange(
            y_ticks[0],
            y_ticks[-1]+1,
            round((y_ticks[-1])/(len(y_tick_labels)-1))
        ),
        y_tick_labels
        )
    ax.yaxis.set_minor_locator(mpl.ticker.MultipleLocator(1))
    # ax.set_aspect(0.8)

    boxes = []
    colors = []
    for i, color in enumerate(set_background):
        if len(color) == 0:
            continue

        boxes.append(
            Rectangle(
                (2*i, 0),
                width=2,
                height=len(y_ticks),
                ls='',
                lw=None,
            )
        )
        colors.append(color)
    ax.add_collection(
        PatchCollection(
            boxes,
            facecolor=colors,
            edgecolor='None',
            alpha=0.2,
        )
    )

    boxes = []
    colors = []
    for i, (xtick, color) in enumerate(zip(x_ticks, set_colors)):
        _d = hist.loc[hist[data_column] == xtick]
        if hist_split is not None:
            split_d = hist_split.loc[hist_split[data_column] == xtick]

        for N in y_ticks:
            locx = x_to_xtick(xtick, x_ticks=x_ticks)
            count = _d[N].iloc[0]
            if np.isnan(count):
                count = 0

            boxes.append(
                Rectangle(
                    (locx, N - 0.5),
                    width=-count,
                    height=1,
                    ls='',
                    lw=0,
                )
            )
            colors.append(color)
            
            if hist_split is None:
                boxes.append(
                    Rectangle(
                        (locx, N - 0.5),
                        width=count,
                        height=1,
                        ls='',
                        lw=0,
                    )
                )
                colors.append(color)
            else:
                count = split_d[N].iloc[0]
                boxes.append(
                    Rectangle(
                        (locx, N - 0.5),
                        width=count,
                        height=1,
                        ls='',
                        lw=0,
                    )
                )
                colors.append(split_color)
        
        # plot mean as hline
        if means is not None and len(means):
            xtick = x_ticks[i]
            mean = means[i]
            count = _d[round(mean)].iloc[0]
            if means_split is None:
                xmax = 2*i + 1 + count
            else:
                xmax = 2*i + 1

            ax.hlines(
                y=mean,
                xmin=2*i + 1 - count,
                xmax=xmax,
                color='black',
                linestyles='dashed',
                linewidths=linewidth
            )

        if means_split is not None and len(means_split):
            xtick = x_ticks[i]
            mean = means_split[i]
            count = split_d[round(mean)].iloc[0]
            ax.hlines(
                y=mean,
                xmin=2*i + 1,
                xmax=2*i + 1 + count,
                color='black',
                linestyles='dashed',
                linewidths=linewidth
            )


    pc = PatchCollection(boxes, facecolor=colors)
    histogram = ax.add_collection(pc)
    # legend_elements = [
    #     mpl.patches.Patch(label='stage: {}'.format(stage)),
    #     # mpl.patches.Patch(facecolor='orange', edgecolor='orange', label='Theory')
    # ]
    # if label is not None:
    #     print(label)
    #     if type(label) is not str:
    #         label = 'E14'
    #     # NOTE weird error
    #     ax.set_title('{:s}'.format(str(label)), x=0.85, y=0.8, fontsize='small')
    # ax.legend(title='stage: {}'.format(stage), loc='upper right')
    # ax.legend(handles=legend_elements, loc='upper right')
    ax.set_ylabel('Neighbour numbers')
    ax.set_ylim(0, len(y_ticks)-1)


def histplot_violin_col(*, hist, col_labels, col_axes, col_column, means=None, means_split=None, **kwargs):
    hist = hist.reset_index()
    if means_split is None:
        means_split = []
        for a in col_axes:
            means_split.append(None)
    if means is None:
        means = []
        for a in col_axes:
            means.append(None)
    for (col, ax, _means, _means_split) in zip(col_labels, col_axes, means, means_split):
        histplot_violin(
            hist.loc[hist[col_column] == col], 
            ax=ax,
            label=col, 
            means=_means, means_split=_means_split,
            **kwargs
        )


In [44]:
for position in ['25I', '50I', '75I']:
    figsize=[6.7, 1.9]
    fig, axes = plt.subplots(1, 4, figsize=figsize, sharex=True, sharey=True)
    (ax1, ax2, ax3, ax4) = axes

    for ax in axes:
        for axis in ['bottom','left']:
            ax.spines[axis].set_linewidth(1./4)
        for axis in ['top','right']:
            ax.spines[axis].set_linewidth(0)
        ax.tick_params(which='both', width=1./4)

    stages = [
        'E8',
        'E10',
        'E12',
        'E14',
    ]

    y_min = 0
    y_max=12
    y_ticks = np.arange(0, y_max+1)
    y_tick_labels = [
        '0',# '',
        '3',# '',
        '6',# '',
        '9',# '',
        '12',
    ]

    data = cells.loc[cells['position'] == position]

    hc = data.loc[data['is_HC']].copy()
    sc = data.loc[~data['is_HC']].copy()


    # HC neighborhood
    hc_norm = hc.groupby(by='stage')['num_neighbors'].count()

    _hist = [hc.loc[hc['num_neighbors'] == i].groupby(by='stage')['num_neighbors'].count() for i in y_ticks]
    _hist = pd.concat(_hist, axis=1, keys=y_ticks)
    _hist = _hist.divide(hc_norm, axis=0)
    _hist['type'] = 'H'

    hist = _hist

    ### HH
    _hist = [hc.loc[hc['num_hair_neighbors'] == i].groupby(by='stage')['num_hair_neighbors'].count() for i in y_ticks]
    _hist = pd.concat(_hist, axis=1, keys=y_ticks)
    _hist = _hist.divide(hc_norm, axis=0)
    _hist['type'] = 'HH'

    hist = pd.concat([hist, _hist], axis=0, join='outer')

    ### HS
    hc['num_support_neighbors'] = hc['num_neighbors'] - hc['num_hair_neighbors']
    _hist = [hc.loc[hc['num_support_neighbors'] == i].groupby(by='stage')['num_support_neighbors'].count() for i in y_ticks]
    _hist = pd.concat(_hist, axis=1, keys=y_ticks)
    _hist = _hist.divide(hc_norm, axis=0)
    _hist['type'] = 'HS'

    hist = pd.concat([hist, _hist], axis=0, join='outer')


    # SC neighborhood
    sc_norm = sc.groupby(by='stage')['num_neighbors'].count()

    _hist = [sc.loc[sc['num_neighbors'] == i].groupby(by='stage')['num_neighbors'].count() for i in y_ticks]
    _hist = pd.concat(_hist, axis=1, keys=y_ticks)
    _hist = _hist.divide(sc_norm, axis=0)
    _hist['type'] = 'S'

    hist = pd.concat([hist, _hist], axis=0, join='outer')

    # SH
    _hist = [sc.loc[sc['num_hair_neighbors'] == i].groupby(by='stage')['num_hair_neighbors'].count() for i in y_ticks]
    _hist = pd.concat(_hist, axis=1, keys=y_ticks)
    _hist = _hist.divide(sc_norm, axis=0)
    _hist['type'] = 'SH'

    hist = pd.concat([hist, _hist], axis=0, join='outer')

    # SS
    sc['num_support_neighbors'] = sc['num_neighbors'] - sc['num_hair_neighbors']
    _hist = [sc.loc[sc['num_support_neighbors'] == i].groupby(by='stage')['num_support_neighbors'].count() for i in y_ticks]
    _hist = pd.concat(_hist, axis=1, keys=y_ticks)
    _hist = _hist.divide(sc_norm, axis=0)
    _hist['type'] = 'SS'


    # concat
    hist = pd.concat([hist, _hist], axis=0, join='outer')
    hist = hist.reset_index()
    hist = hist.set_index(['stage', 'type'])

    hist = hist.reset_index()


    # fig, axes = plt.subplots(1, 4, figsize=figsize, sharex=True, sharey=True)
    # (ax1, ax2, ax3, ax4) = axes

    for ax in axes:
        for axis in ['bottom','left']:
            ax.spines[axis].set_linewidth(1./4)
        for axis in ['top','right']:
            ax.spines[axis].set_linewidth(0)
        ax.tick_params(which='both', width=1./4)

    x_ticks = [
        'H',
        'HH',
        # 'HS',
        'S',
        'SH',
        'SS'
    ]
    x_tick_labels = [
        'HC (all)',
        'HC-HC',
        #'', 'HC-SC',
        'SC (all)', 
        'SC-HC',
        'SC-SC',
    ]


    set_colors = [
        'red',
        'orangered',
        # 'gold',
        'dimgrey',
        'darkgray',
        'silver'
    ]

    set_background = [
        # 'coral',
        # '',
        # 'gray',
    ]

    means = []

    for stage in ['E8', 'E10', 'E12', 'E14']:
        mean_data = data.loc[data['stage'] == stage]
        tmp = [
            mean_data.loc[mean_data['is_HC']]['num_neighbors'].mean(),
            mean_data.loc[mean_data['is_HC']]['num_hair_neighbors'].mean(),
            mean_data.loc[~mean_data['is_HC']]['num_neighbors'].mean(),
            mean_data.loc[~mean_data['is_HC']]['num_hair_neighbors'].mean(),
            mean_data.loc[~mean_data['is_HC']]['num_neighbors'].mean() - mean_data.loc[~mean_data['is_HC']]['num_hair_neighbors'].mean(),
        ]
        means.append(tmp)

    histplot_violin_col(
        hist=hist,
        col_column='stage',
        col_labels=stages,
        col_axes=[ax1, ax2, ax3, ax4],
        x_ticks=x_ticks,
        x_tick_labels=x_tick_labels,
        y_ticks=y_ticks,
        y_tick_labels=y_tick_labels,
        set_colors=set_colors,
        data_column='type',
        set_background=set_background,
        means=means,
    )


    fig.tight_layout()

    plt.savefig(f"{figure_path}/neighborhood__{position}.svg")


### Neighbourhood perturbation histograms

In [45]:
def plot_hist(*, data, x, discrete_bins=False,  **kwargs):
    ax = plt.gca()

    sns.histplot(
        data=data,
        x=x,
        hue='is_HC',
        palette=['gray', 'red'],
        multiple='stack',
        stat='probability',
        # common_norm=True,
        legend=False,
        alpha=0.5,
        **kwargs
    )

    __hc = data.loc[data['is_HC']]
    hc_mean = data.loc[data['is_HC'], x].mean()
    sc_mean = data.loc[~data['is_HC'], x].mean()

    vlines_kwargs = dict({
        'lw': linewidth,
        'ls': 'dashed',
        'colors': ['black', 'red', 'gray'],
    })

    if discrete_bins:
        __mean = round(data[x].mean())
        __hc_mean = round(hc_mean)
        __sc_mean = round(sc_mean)

        plt.vlines(
            x=[
                data[x].mean(),
                hc_mean,
                sc_mean,
            ],
            ymin=[
                0,
                0,
                __hc.loc[np.round(__hc[x])==np.round(__sc_mean), x].count() / data[x].count(),
            ],
            ymax=[
                data.loc[np.round(data[x])==np.round(__mean), x].count() / data[x].count(),
                __hc.loc[np.round(__hc[x])==np.round(__hc_mean), x].count() / data[x].count(),
                data.loc[np.round(data[x])==np.round(__sc_mean), x].count() / data[x].count(),
            ],
            **vlines_kwargs
        )
    else:
        ymax = ax.get_ylim()[1]
        plt.vlines(
            x=[
                data[x].mean(),
                hc_mean,
                sc_mean,
            ],
            ymin=[
                0,
                0,
                0.,
            ],
            ymax=[
                ymax,
                ymax,
                ymax
            ],
            **vlines_kwargs
        )

        

    legend = ax.legend(
        labels=['all','HC', 'SC'],
    )

In [46]:
treatments = [
    'Control',

    'MLCKdmso-4hrs',
    'MLCKdmso-16hrs',
    'MLCKml7-4hrs',
    'MLCKml7-16hrs',

    'Vangl2Control',
    'Vangl2KO',

    'Atoh1control',
    'Atoh1KO',

    'ROCKcontrol-4hrs',
    'ROCKcontrol-16hrs',
    'ROCKy27632-4hrs',
    'ROCKy27632-16hrs',

    'PTXcontrol',
    'PTXinhibition',
]
descriptions = [
    'Fixed BP (Control, 25I @ E10)',

    'MLCK (DMSO, 4hrs)',
    'MLCK (DMSO, 16hrs)',
    'MLCK (4hrs)',
    'MLCK (washoff, 16hrs)',

    'Vangl2 (Control)',
    'Vangl2-KO',

    'Atoh1 (Control)',
    'Atoh1-KO',

    'ROCK (DMSO, 4hrs)',
    'ROCK (DMSO, 16hrs)',
    'ROCK (4hrs)',
    'ROCK (washoff, 16hrs)',

    'PTX (DMSO)',
    'PTX (4hrs)',
]

bins = [
    2.6, 3.4,
    3.6, 4.4,
    4.6, 5.4,
    5.6, 6.4,
    6.6, 7.4,
    7.6, 8.4,
    8.6, 9.4,
    9.6, 10.4,
    10.6
]

try: 
    os.mkdir(f"{figure_path}/neighbors_hc_vs_sc")
except OSError as error: 
    print(error)
try: 
    os.mkdir(f"{figure_path}/neighbors_hh_contacts")
except OSError as error: 
    print(error)
try: 
    os.mkdir(f"{figure_path}/neighbors_hc")
except OSError as error: 
    print(error)


for treatment, description in zip(treatments, descriptions):
    print(treatment)
    data = cells_experiments.loc[cells_experiments['Treatment'] == treatment].copy()
    data = data.loc[data['stage']=='E10']
    data = data.loc[data['position']=='25I']

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=[2*5.9/3,5.9/4*3/2])

    plt.sca(ax1)
    plot_hist(data=data, x='normalized_area_cells', discrete = False)
    ax1.set_xlim(0, 3)

    plt.sca(ax2)
    plot_hist(data=data, x=f'num_neighbors', discrete = True, discrete_bins=True)
    ax2.set_xlim(3, 10)

    ax2.set_xlim([3, 10])
    ax2.set_xticks([4, 6, 8, 10])
    ax2.xaxis.set_minor_locator(mpl.ticker.AutoMinorLocator(2))
    ax2.set_xlabel('Neighbour number')

    plt.suptitle(f'{description}')

    plt.savefig(f"{figure_path}/neighbors_hc_vs_sc/{treatment}.svg")
    ax1.clear()
    ax2.clear()
    plt.close()

    
    fig, ax = plt.subplots(1, 1, figsize=[5.9/4,5.9/4*3/2])

    plt.sca(ax)
    sns.histplot(
        data=data,
        x='num_neighbors',
        bins=bins,
        # discrete=True,
        hue='is_HC',
        palette=['gray', 'red'],
        multiple='dodge',
        stat='probability',
        common_norm=False,
        legend=False,
        alpha=0.5
    )
    ax.set_xlim([2.5, 10])
    ax.set_xticks([4, 6, 8, 10])
    ax.xaxis.set_minor_locator(mpl.ticker.AutoMinorLocator(2))
    ax.set_xlabel('Neighbour number')

    ax.set_ylim([0, 0.5])
    ax.set_yticks([0, 0.2, 0.4])
    ax.yaxis.set_minor_locator(mpl.ticker.AutoMinorLocator(2))

    plt.suptitle(f'{description}')

    plt.savefig(f"{figure_path}/neighbors_hc_vs_sc/{treatment}__dodge.svg")
    plt.close()

    fig, ax = plt.subplots(1, 1, figsize=[5.9/4,5.9/4*3/2])


    plot_hist(data=data, x=f'num_neighbors__corr', discrete=False, discrete_bins=True, bins=np.arange(2.5, 10.5, 1))
    ax.set_xlim(3, 10)

    ax.set_xlim([3, 10])
    ax.set_xticks([4, 6, 8, 10])
    # ax.set_xticks([3, 6, 9])
    ax.xaxis.set_minor_locator(mpl.ticker.AutoMinorLocator(2))
    ax.set_xlabel('corr. Neighbour number')

    ax.set_ylim([0, 0.4])
    ax.set_yticks([0., 0.2, 0.4])

    plt.suptitle(f'{description}')
    plt.savefig(f"{figure_path}/neighbors_hc_vs_sc/{treatment}__corrected.svg")
    plt.close()


    fig, ax = plt.subplots(1, 1, figsize=[5.9/4,5.9/4*3/2])

    sns.histplot(
        data=data.loc[data['is_HC']],
        x='num_hair_neighbors',
        bins=[
            -0.4, 0.4,
            0.6, 1.4,
            1.6, 2.4,
            2.6, 3.4,
            3.6, 4.4
        ],
        hue='is_HC',
        palette=['red'],
        multiple='dodge',
        stat='probability',
        common_norm=False,
        legend=False,
        alpha=0.5
    )

    ax.set_xlim([-0.5, 4.5])
    ax.set_xticks([0, 1, 2, 3, 4])
    ax.set_xlabel('Neighbour number')

    ax.set_ylim([0, 1.])
    ax.set_yticks([0., 0.5, 1.])
    ax.yaxis.set_minor_locator(mpl.ticker.AutoMinorLocator(2))

    plt.suptitle(f'{description}')
    plt.savefig(f"{figure_path}/neighbors_hh_contacts/{treatment}.svg")
    plt.close()

[Errno 17] File exists: '/home/julian/image_analysis/analysis/1_figures/neighbors_hc_vs_sc'
[Errno 17] File exists: '/home/julian/image_analysis/analysis/1_figures/neighbors_hh_contacts'
[Errno 17] File exists: '/home/julian/image_analysis/analysis/1_figures/neighbors_hc'
Control
MLCKdmso-4hrs
MLCKdmso-16hrs
MLCKml7-4hrs
MLCKml7-16hrs
Vangl2Control
Vangl2KO
Atoh1control
Atoh1KO
ROCKcontrol-4hrs
ROCKcontrol-16hrs
ROCKy27632-4hrs
ROCKy27632-16hrs
PTXcontrol
PTXinhibition


In [47]:
figsizes=dict({
    'Vangl2': [1.6, 1.475],
    'MLCK': [1.3, 1.3],
    'ROCK': [1.4, 1.2],
    'ROCK_16hrs': [1.4, 1.2],
    'MLCK_16hrs': [1.4, 1.2],
    'PTX': [1.6, 1.4]
})

data = cells_experiments.copy()

tmp = data.loc[(data['Experiment_Series'] == 'MLCK') & (~data['is_Control'])].copy()
tmp['Experiment'] = 'MLCK_inhibition'
tmp['Experiment_Series'] = 'MLCK_recovery'
tmp['is_Control'] = True
data = pd.concat([data, tmp])

tmp = data.loc[(data['Experiment_Series'] == 'MLCK_16hrs') & (~data['is_Control'])].copy()
tmp['Experiment'] = 'MLCK_recovery'
tmp['Experiment_Series'] = 'MLCK_recovery'
tmp['is_Control'] = False
data = pd.concat([data, tmp])

tmp = data.loc[(data['Experiment_Series'] == 'ROCK') & (~data['is_Control'])].copy()
tmp['Experiment'] = 'ROCK_inhibition'
tmp['Experiment_Series'] = 'ROCK_recovery'
tmp['is_Control'] = True
data = pd.concat([data, tmp])

tmp = data.loc[(data['Experiment_Series'] == 'ROCK_16hrs') & (~data['is_Control'])].copy()
tmp['Experiment'] = 'ROCK_recovery'
tmp['Experiment_Series'] = 'ROCK_recovery'

tmp['is_Control'] = False
data = pd.concat([data, tmp])


y_min = 0
y_max=9
y_ticks = np.arange(0, y_max+1)
y_tick_labels = [
    '0',# '',
    '3',# '',
    '6',# '',
    '9',# '',
    # '12',
]

try: 
    os.mkdir(f"{figure_path}/neighborhood")
except OSError as error: 
    print(error)   

for Series in data['Experiment_Series'].unique():
    if Series == 'Control':
        continue

    print("\n")
    print(Series)

    for num_neighbors, description in zip(['num_neighbors', 'num_neighbors__corr'], ['', '__corrected']):
        if Series in figsizes:
            figsize=figsizes[Series]
        else:
            figsize=[2., 2.]
        fig, ax = plt.subplots(1, 1, figsize=figsize, sharex=True, sharey=True)

        for axis in ['bottom','left']:
            ax.spines[axis].set_linewidth(1./4)
        for axis in ['top','right']:
            ax.spines[axis].set_linewidth(0)
        ax.tick_params(which='both', width=1./4)


        histograms = dict()
        means = dict()
        for is_Control in [True, False]:
            d = data.loc[
                # (data['position'] == position) &
                # (data['stage'] == stage) &
                (data['Experiment_Series'] == Series) &
                (data['is_Control'] == is_Control)
            ]

            hc = d.loc[d['is_HC']].copy()
            sc = d.loc[~d['is_HC']].copy()


            # HC neighborhood
            hc_norm = hc.groupby(by='stage')[num_neighbors].count()

            _hist = [hc.loc[(hc[num_neighbors] > i - 0.5) & (hc[num_neighbors] < i + 0.5)].groupby(by='stage')[num_neighbors].count() for i in y_ticks]
            _hist = pd.concat(_hist, axis=1, keys=y_ticks)
            _hist = _hist.divide(hc_norm, axis=0)
            _hist['type'] = 'H'

            hist = _hist

            ### HH
            _hist = [hc.loc[hc['num_hair_neighbors'] == i].groupby(by='stage')['num_hair_neighbors'].count() for i in y_ticks]
            _hist = pd.concat(_hist, axis=1, keys=y_ticks)
            _hist = _hist.divide(hc_norm, axis=0)
            _hist['type'] = 'HH'

            hist = pd.concat([hist, _hist], axis=0, join='outer')

            ### HS
            hc['num_support_neighbors'] = hc[num_neighbors] - hc['num_hair_neighbors']
            _hist = [hc.loc[(hc['num_support_neighbors'] > i - 0.5) & (hc['num_support_neighbors'] < i + 0.5)].groupby(by='stage')['num_support_neighbors'].count() for i in y_ticks]
            _hist = pd.concat(_hist, axis=1, keys=y_ticks)
            _hist = _hist.divide(hc_norm, axis=0)
            _hist['type'] = 'HS'

            hist = pd.concat([hist, _hist], axis=0, join='outer')


            # SC neighborhood
            sc_norm = sc.groupby(by='stage')[num_neighbors].count()

            _hist = [sc.loc[(sc[num_neighbors] > i - 0.5) & (sc[num_neighbors] < i + 0.5)].groupby(by='stage')[num_neighbors].count() for i in y_ticks]
            _hist = pd.concat(_hist, axis=1, keys=y_ticks)
            _hist = _hist.divide(sc_norm, axis=0)
            _hist['type'] = 'S'

            hist = pd.concat([hist, _hist], axis=0, join='outer')

            # SH
            _hist = [sc.loc[sc['num_hair_neighbors'] == i].groupby(by='stage')['num_hair_neighbors'].count() for i in y_ticks]
            _hist = pd.concat(_hist, axis=1, keys=y_ticks)
            _hist = _hist.divide(sc_norm, axis=0)
            _hist['type'] = 'SH'

            hist = pd.concat([hist, _hist], axis=0, join='outer')

            # SS
            sc['num_support_neighbors'] = sc[num_neighbors] - sc['num_hair_neighbors']
            _hist = [sc.loc[(sc['num_support_neighbors'] > i - 0.5) & (sc['num_support_neighbors'] < i + 0.5)].groupby(by='stage')['num_support_neighbors'].count() for i in y_ticks]
            _hist = pd.concat(_hist, axis=1, keys=y_ticks)
            _hist = _hist.divide(sc_norm, axis=0)
            _hist['type'] = 'SS'


            # concat
            hist = pd.concat([hist, _hist], axis=0, join='outer')
            hist = hist.reset_index()
            hist = hist.set_index(['stage', 'type'])

            hist = hist.reset_index()
            histograms[is_Control] = hist

            means[is_Control] = [[
                hc[num_neighbors].mean(),
                hc['num_hair_neighbors'].mean(),
                sc[num_neighbors].mean(),
                sc['num_hair_neighbors'].mean(),
                sc[num_neighbors].mean() - sc['num_hair_neighbors'].mean(),
            ]]

        x_ticks = [
        'H',
        'HH',
        # 'HS',
        'S',
        'SH',
        'SS'
        ]
        x_tick_labels = [
        'HC (all)',
        'HC-HC',
        #'', 'HC-SC',
        'SC (all)', 
        'SC-HC',
        'SC-SC',
        ]


        set_colors = [
        'red',
        'orangered',
        # 'gold',
        'dimgrey',
        'darkgray',
        'silver'
        ]

        set_background = [
        # 'coral',
        # '',
        # 'gray',
        ]

        histplot_violin_col(
            hist=histograms[True],
            col_column='stage',
            col_labels=['E10'],
            col_axes=[ax],
            x_ticks=x_ticks,
            x_tick_labels=x_tick_labels,
            y_ticks=y_ticks,
            y_tick_labels=y_tick_labels,
            set_colors=set_colors,
            data_column='type',
            set_background=set_background,
            hist_split=histograms[False],
            means=means[True],
            means_split=means[False],
            split_color='blue',
        )

        if len(description) == 0:
            print("HC-HC contacts in ", Series, ": p =",
                "%.2E" % scipy.stats.ttest_ind(
                    data.loc[
                        (data['Experiment_Series'] == Series) &
                        (data['is_Control'] == True) &
                        (data['is_HC']),
                        'num_hair_neighbors'
                    ],
                    data.loc[
                        (data['Experiment_Series'] == Series) &
                        (data['is_Control'] == False) &
                        (data['is_HC']),
                        'num_hair_neighbors'
                    ],
                    equal_var=False
                )[1],
                "(",
                data.loc[
                    (data['Experiment_Series'] == Series) &
                    (data['is_Control'] == True) &
                    (data['is_HC']),
                    'num_hair_neighbors'
                ].sum(),
                "vs",
                data.loc[
                    (data['Experiment_Series'] == Series) &
                    (data['is_Control'] == False) &
                    (data['is_HC']),
                    'num_hair_neighbors'
                ].sum(), 
                ")\n"
            )
            print("number of HC-HC contacts")
            print(
                data.loc[
                    (data['Experiment_Series'] == Series) &
                    (data['is_HC']),
                ].groupby(by=['is_Control', 'num_hair_neighbors'])['num_hair_neighbors'].count())
            print()
        else:
            print("SC-HC neighbour distributions in ", Series, "(", description, "): p =",
                "%.2E" % scipy.stats.ttest_ind(
                    data.loc[
                        (data['Experiment_Series'] == Series) &
                        (data['is_Control'] == True) &
                        (data['is_HC'] == False),
                        'num_hair_neighbors'
                    ],
                    data.loc[
                        (data['Experiment_Series'] == Series) &
                        (data['is_Control'] == False) &
                        (data['is_HC'] == False),
                        'num_hair_neighbors'
                    ],
                    equal_var=True
                )[1]
            )
            print("SC-SC neighbour distributions in ", Series, "(", description, "): p =",
                "%.2E" % scipy.stats.ttest_ind(
                    data.loc[
                        (data['Experiment_Series'] == Series) &
                        (data['is_Control'] == True) &
                        (data['is_HC'] == False),
                        num_neighbors
                    ]
                    -
                    data.loc[
                        (data['Experiment_Series'] == Series) &
                        (data['is_Control'] == True) &
                        (data['is_HC'] == False),
                        'num_hair_neighbors'
                    ],
                    data.loc[
                        (data['Experiment_Series'] == Series) &
                        (data['is_Control'] == False) &
                        (data['is_HC'] == False),
                        num_neighbors
                    ]
                    -
                    data.loc[
                        (data['Experiment_Series'] == Series) &
                        (data['is_Control'] == False) &
                        (data['is_HC'] == False),
                        'num_hair_neighbors'
                    ],
                    equal_var=True
                )[1]
            )
        print("HC neighbour distributions in ", Series, "(", description, "): p =",
            "%.2E" % scipy.stats.ttest_ind(
                data.loc[
                    (data['Experiment_Series'] == Series) &
                    (data['is_Control'] == True) &
                    (data['is_HC']),
                    num_neighbors
                ],
                data.loc[
                    (data['Experiment_Series'] == Series) &
                    (data['is_Control'] == False) &
                    (data['is_HC']),
                    num_neighbors
                ],
                equal_var=True
            )[1]
        )
        print("SC neighbour distributions in ", Series, "(", description, "): p =",
            "%.2E" % scipy.stats.ttest_ind(
                data.loc[
                    (data['Experiment_Series'] == Series) &
                    (data['is_Control'] == True) &
                    (data['is_HC'] == False),
                    num_neighbors
                ],# ax.set_ylim([0, 3050])

                data.loc[
                    (data['Experiment_Series'] == Series) &
                    (data['is_Control'] == False) &
                    (data['is_HC'] == False),
                    num_neighbors
                ],
                equal_var=True
            )[1]
        )
        print(f"HC vs SC distribution in {Series} control ({description}): p =",
            "%.2E" % scipy.stats.ttest_ind(
                data.loc[
                    (data['Experiment_Series'] == Series) &
                    (data['is_Control'] == True) &
                    (data['is_HC'] == True),
                    num_neighbors
                ],
                data.loc[
                    (data['Experiment_Series'] == Series) &
                    (data['is_Control'] == True) &
                    (data['is_HC'] == False),
                    num_neighbors
                ],
                equal_var=True
            )[1]
        )
        print(f"HC vs SC distribution in {Series} ({description}): p =",
            "%.2E" % scipy.stats.ttest_ind(
                data.loc[
                    (data['Experiment_Series'] == Series) &
                    (data['is_Control'] == False) &
                    (data['is_HC'] == True),
                    num_neighbors
                ],
                data.loc[
                    (data['Experiment_Series'] == Series) &
                    (data['is_Control'] == False) &
                    (data['is_HC'] == False),
                    num_neighbors
                ],
                equal_var=True
            )[1]
        )

        plt.savefig(f"{figure_path}/neighborhood/{Series}{description}.svg")
        plt.close()


[Errno 17] File exists: '/home/julian/image_analysis/analysis/1_figures/neighborhood'


Vangl2
HC-HC contacts in  Vangl2 : p = 2.49E-10 ( 0 vs 50 )

number of HC-HC contacts
is_Control  num_hair_neighbors
False       0                     706
            1                      42
            2                       1
            3                       2
True        0                     114
Name: num_hair_neighbors, dtype: int64

HC neighbour distributions in  Vangl2 (  ): p = 7.91E-01
SC neighbour distributions in  Vangl2 (  ): p = 8.91E-02
HC vs SC distribution in Vangl2 control (): p = 3.92E-62
HC vs SC distribution in Vangl2 (): p = 9.88E-304
SC-HC neighbour distributions in  Vangl2 ( __corrected ): p = 7.41E-10
SC-SC neighbour distributions in  Vangl2 ( __corrected ): p = 3.67E-11
HC neighbour distributions in  Vangl2 ( __corrected ): p = 1.28E-04
SC neighbour distributions in  Vangl2 ( __corrected ): p = 3.27E-01
HC vs SC distribution in Vangl2 control (__corrected): p = 4.97E-5

In [48]:
Series = 'Atoh1'
print(np.unique(data['Experiment_Series']))
print(data.loc[
                (data['Experiment_Series'] == Series) &
                (data['is_Control'] == True) &
                (data['is_HC'] == False),
                'num_hair_neighbors'
            ])
print(data.loc[
                (data['Experiment_Series'] == Series) &
                (data['is_Control'] == False) &
                (data['is_HC'] == False),
                'num_hair_neighbors'
            ])
print("SC-HC neighbour distributions in ", Series, "(", description, "): p =",
        "%.2E" % scipy.stats.ttest_ind(
            data.loc[
                (data['Experiment_Series'] == Series) &
                (data['is_Control'] == True) &
                (data['is_HC'] == False),
                'num_hair_neighbors'
            ],
            data.loc[
                (data['Experiment_Series'] == Series) &
                (data['is_Control'] == False) &
                (data['is_HC'] == False),
                'num_hair_neighbors'
            ],
            # equal_var=True
        )[1]
    )

['Atoh1' 'Control' 'MLCK' 'MLCK_16hrs' 'MLCK_recovery' 'PTX' 'ROCK'
 'ROCK_16hrs' 'ROCK_recovery' 'Vangl2']
41311    3
41312    2
41313    3
41314    3
41316    2
        ..
42937    1
42938    2
42939    2
42940    2
42941    2
Name: num_hair_neighbors, Length: 995, dtype: int64
42983    1
42984    2
42985    2
42986    0
42987    0
        ..
45740    0
45741    1
45744    0
45748    0
45750    0
Name: num_hair_neighbors, Length: 2059, dtype: int64
SC-HC neighbour distributions in  Atoh1 ( __corrected ): p = 0.00E+00


In [49]:
for Series in ['MLCK_recovery', 'ROCK_recovery']:
    fig, ax = plt.subplots(1, 1, figsize=[1.4, 1.2])

    d = data.loc[data['Experiment_Series'] == Series]

    sns.histplot(
        data=d.loc[d['is_HC']],
        x='num_hair_neighbors',
        hue='Experiment',
        # hue_order=[True, False],
        multiple='dodge',
        palette='tab10',
        stat='probability',
        common_norm=False,
        bins=[-0.4, 0.4, 0.6, 1.4, 1.6, 2.4, 2.6, 3.4, 3.6, 4.4]
    )

    ax.set_xlim([-0.5, 4.5])
    ax.set_xticks(range(5))
    ax.set_xlabel("Number of HCs around each HC")

    ax.set_ylim([0., 1])
    ax.set_yticks([0., 0.5, 1.])
    ax.yaxis.set_minor_locator(mpl.ticker.AutoMinorLocator(2))

    plt.savefig(f"{figure_path}/HC_contacts__{Series}.svg")


In [50]:
fig, ax = plt.subplots(1, 1)

data = cells_experiments.loc[
    (
        (cells_experiments['Experiment_Series']=='MLCK') | 
        (cells_experiments['Experiment_Series']=='MLCK_16hrs')
    ) &
    (cells_experiments['is_HC'])
]

sns.violinplot(
    data=data,
    x='Experiment_Series',
    hue='is_Control', hue_order=[True, False],
    split=True,
    y='num_hair_neighbors',
)

ax.set_ylim([0, 3])


box_pairs=[
    (('MLCK', True), ('MLCK', False)),
    (('MLCK', True), ('MLCK_16hrs', True)),
    (('MLCK', False), ('MLCK_16hrs', False)),
    (('MLCK_16hrs', True), ('MLCK_16hrs', False))
]

annotator = Annotator(
    ax,
    box_pairs,
    data=data,
    x='Experiment_Series',
    hue='is_Control', hue_order=[True, False],
    split=True,
    y='num_hair_neighbors',
)
annotator.configure(
    test='Mann-Whitney',
    line_width=0.75,
)
annotator.apply_and_annotate()

/tmp/ipykernel_79459/1482425149.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax = plt.subplots(1, 1)
/home/julian/.local/lib/python3.10/site-packages/statannotations/_Plotter.py:337: UserWarning: Invalid x-position found. Are the same parameters passed to seaborn and statannotations calls? or are there few data points?
  warnings.warn(


p-value annotation legend:
      ns: p <= 1.00e+00
       *: 1.00e-02 < p <= 5.00e-02
      **: 1.00e-03 < p <= 1.00e-02
     ***: 1.00e-04 < p <= 1.00e-03
    ****: p <= 1.00e-04

MLCK_16hrs_True vs. MLCK_16hrs_False: Mann-Whitney-Wilcoxon test two-sided, P_val:1.422e-01 U_stat=2.903e+04
MLCK_True vs. MLCK_False: Mann-Whitney-Wilcoxon test two-sided, P_val:6.994e-09 U_stat=6.784e+03
MLCK_True vs. MLCK_16hrs_True: Mann-Whitney-Wilcoxon test two-sided, P_val:1.348e-01 U_stat=1.811e+04
MLCK_False vs. MLCK_16hrs_False: Mann-Whitney-Wilcoxon test two-sided, P_val:3.935e-08 U_stat=1.697e+04


(<AxesSubplot:xlabel='Experiment_Series', ylabel='num_hair_neighbors'>,
  <statannotations.Annotation.Annotation at 0x7f57f1ed7a60>])

In [51]:
fig, ax = plt.subplots(1, 1)

data = cells_experiments.loc[
    (
        (cells_experiments['Experiment_Series']=='ROCK') | 
        (cells_experiments['Experiment_Series']=='ROCK_16hrs')
    ) &
    (cells_experiments['is_HC'])
]

sns.violinplot(
    data=data,
    x='Experiment_Series',
    order=['ROCK', 'ROCK_16hrs'],
    hue='is_Control', hue_order=[True, False],
    split=True,
    y='num_hair_neighbors',
    # hue='is_Control',
    # hue_order=[True, False],
)

ax.set_ylim([0, 3])


box_pairs=[
    (('ROCK', True), ('ROCK', False)),
    (('ROCK', True), ('ROCK_16hrs', True)),
    (('ROCK', False), ('ROCK_16hrs', False)),
    (('ROCK_16hrs', True), ('ROCK_16hrs', False))
    # ((True, 'Vangl2Control'), (True, 'Vangl2KO')),
    # ((False, 'Vangl2Control'), (False, 'Vangl2KO')),
]

annotator = Annotator(
    ax,
    box_pairs,
    data=data,
    x='Experiment_Series',
    order=['ROCK', 'ROCK_16hrs'],
    hue='is_Control', hue_order=[True, False],
    split=True,
    y='num_hair_neighbors',
)
annotator.configure(
    test='Mann-Whitney',
    line_width=0.75,
)
annotator.apply_and_annotate()

/home/julian/.local/lib/python3.10/site-packages/statannotations/_Plotter.py:337: UserWarning: Invalid x-position found. Are the same parameters passed to seaborn and statannotations calls? or are there few data points?
  warnings.warn(
/home/julian/.local/lib/python3.10/site-packages/statannotations/_Plotter.py:337: UserWarning: Invalid x-position found. Are the same parameters passed to seaborn and statannotations calls? or are there few data points?
  warnings.warn(


p-value annotation legend:
      ns: p <= 1.00e+00
       *: 1.00e-02 < p <= 5.00e-02
      **: 1.00e-03 < p <= 1.00e-02
     ***: 1.00e-04 < p <= 1.00e-03
    ****: p <= 1.00e-04

ROCK_16hrs_True vs. ROCK_16hrs_False: Mann-Whitney-Wilcoxon test two-sided, P_val:2.115e-07 U_stat=7.854e+03
ROCK_True vs. ROCK_False: Mann-Whitney-Wilcoxon test two-sided, P_val:8.305e-21 U_stat=5.517e+03
ROCK_True vs. ROCK_16hrs_True: Mann-Whitney-Wilcoxon test two-sided, P_val:2.375e-01 U_stat=1.144e+04
ROCK_False vs. ROCK_16hrs_False: Mann-Whitney-Wilcoxon test two-sided, P_val:2.404e-06 U_stat=1.179e+04


(<AxesSubplot:xlabel='Experiment_Series', ylabel='num_hair_neighbors'>,
  <statannotations.Annotation.Annotation at 0x7f57f0bb6080>])

## Area mapped

In [52]:
def tissue_data_errorbar(
    *,
    x: str='HC_area',
    xlabel: str='Normalised HC area',
    y: str,
    ylabel: str=None,
    hue='Stage',
    palette=palette_stages,
    style='P-D position',
    plot_control: bool=True, plot_errorbar=True,
    plot_treatment: bool=False, plot_errorbar_treatment=True,
    xlim=[0., 4.], ylim=None, xticks=None, yticks=None,
    figsize=[5.9/3, 5.9/4],
    savefig: str=None,
    mask=None,
    mask_treatment=None,
    tight_layout=True,
):  
    fig, ax = plt.subplots(1, 1, figsize=figsize)
    if tight_layout:
        fig.tight_layout()

    data = tissue_data.copy()
    if mask is not None:
        data = tissue_data.loc[mask]

    if plot_control:
        sns.scatterplot(
            data=data,
            x=x,
            y=y,
            hue=hue,
            palette=palette,
            style=style,
            # size='S-I position',
            # size_order=['Superior', 'Mid', 'Inferior']
        )
        if plot_errorbar:
            ax.errorbar(
                x=data[x],
                y=data[y],
                xerr=data["{}__std".format(x) ] if "{}__std".format(x) in data.columns else None,
                yerr=data["{}__std".format(y) ] if "{}__std".format(y) in data.columns else None,
                ls='',
                color='black', 
                linewidth=0.2, alpha=0.4,
                zorder=0
            )

    if plot_treatment:
        __data = tissue_data__experiments.copy()
        if mask_treatment is not None:
            __data = __data.loc[mask_treatment]
        __data = __data.loc[tissue_data__experiments['Experiment'] != 'Control']
        __data['__Experiment'] = __data['Experiment']
        __data['Experiment'] = __data['Experiment_Series']
        style_order=['Live', 'MLCK', 'MLCK_16hrs', 'Vangl2', 'ROCK', 'ROCK_16hrs', 'Atoh1', 'PTX']
        __data.Experiment = __data.Experiment.astype("category")
        __data.Experiment = __data.Experiment.cat.set_categories(style_order)
        sns.scatterplot(
            data=__data,
            x=x,
            y=y,
            hue='is_Control',
            hue_order=[True, False],
            palette=['blue', 'red'],
            style='Experiment',
        )
        if plot_errorbar_treatment:
            ax.errorbar(
                x=__data[x],
                y=__data[y],
                xerr=__data["{}__std".format(x) ] if "{}__std".format(x) in __data.columns else None,
                yerr=__data["{}__std".format(y) ] if "{}__std".format(y) in __data.columns else None,
                ls='',
                color='black', 
                linewidth=0.2, alpha=0.4,
                zorder=0
            )

    ax.set_xlim(xlim)
    ax.set_ylim(ylim)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel if ylabel is not None else y)
    if xticks is not None:
        ax.set_xticks(xticks)
    if yticks is not None:
        ax.set_yticks(yticks)

    mpl.pyplot.legend(bbox_to_anchor=(1,1.025), fontsize='xx-small')


    if savefig is not None:
        plt.savefig(f"{figure_path}/area_mapped__{savefig}.svg")
    else:
        return fig, ax


In [53]:
tissue_data_errorbar(
    y='coordination_number', ylabel='HC neighbour no.',
    ylim=[3, 11],
    savefig='HC_neighbours',
)

In [54]:
tissue_data_errorbar(
    y='subordination_number', ylabel='SC neighbour no.',
    x='SC_area', xlabel='Normalised SC area',
    xlim=[0,1.5], ylim=[3, 8],
    savefig='SC_neighbours__SC_area',
)

In [55]:

np.polyfit(tissue_data['HC_area'], tissue_data['subordination_number'], 1)

m_HC = np.polyfit(tissue_data['HC_area'], tissue_data['coordination_number'], 1)[0]
m_SC = np.polyfit(tissue_data['HC_area'], tissue_data['subordination_number'], 1)[0]
def correct_neighbors(nbs, area, is_HC):
    if is_HC:
        m = m_HC
    else:
        m = m_SC
    return nbs  + m * (1. - area)

tissue_data['coordination_number__corr'] = correct_neighbors(tissue_data['coordination_number'], tissue_data['HC_area'], True)
tissue_data['subordination_number__corr'] = correct_neighbors(tissue_data['subordination_number'], tissue_data['HC_area'], False)
tissue_data['delta_area'] = tissue_data['subordination_number__corr'] - tissue_data['coordination_number__corr']

In [56]:
tissue_data_errorbar(
    y='coordination_number__corr', ylabel='HC neighbour no.',
    ylim=[3, 11],
)

(<Figure size 196.667x147.5 with 1 Axes>,
 <AxesSubplot:xlabel='Normalised HC area', ylabel='HC neighbour no.'>)

In [57]:
tissue_data_errorbar(
    y='delta_area', ylabel='HC neighbour no.',
    # ylim=[3, 11],
)

(<Figure size 196.667x147.5 with 1 Axes>,
 <AxesSubplot:xlabel='Normalised HC area', ylabel='HC neighbour no.'>)

In [58]:
tissue_data_errorbar(
    y='subordination_number__corr', ylabel='HC neighbour no.',
    ylim=[3, 11],
    savefig='total_neighbours',
)

In [59]:
fig, ax = plt.subplots()
sns.scatterplot(data=tissue_data, x='HC_area', y='coordination_number')
ax.plot(np.linspace(0, 4, 101), np.linspace(0, 4, 101) * 1.514 + 3.865)

In [60]:
fig, ax = tissue_data_errorbar(
    y='coordination_number', ylabel='HC neighbour number',
    ylim=[3, 11],
    plot_treatment=True,
    savefig=None,
    tight_layout=True
    # mask_treatment=mask_treatment
)

sns.regplot(
    data=tissue_data,
    x='HC_area',
    y='coordination_number',
    scatter=False,
    x_ci='sd', ci=99,
    line_kws=dict(
        lw=3*linewidth,
        color='black')
)

ax.set_xlabel('norm. HC area')
ax.set_ylabel('HC neighbour number')

plt.savefig(f"{figure_path}/area_mapped__HC_neighbours__w_treatment.svg")


In [61]:
mask_treatment = tissue_data__experiments['Experiment'] == 'Vangl2KO'
mask_treatment = np.where(
    tissue_data__experiments['Experiment'] == 'Vangl2Control',
    True,
    mask_treatment
)
print(np.unique(tissue_data__experiments.loc[mask_treatment]['filename']))
tissue_data_errorbar(
    y='coordination_number', ylabel='HC neighbour no.',
    ylim=[3, 11],
    plot_treatment=True,
    savefig='HC_neighbours__Vangl2KO',
    mask_treatment=mask_treatment
)

['E10_Vangl2Control_1.tif' 'E10_Vangl2Control_2.tif'
 'E10_Vangl2Control_3.tif' 'E10_Vangl2KO_1.tif' 'E10_Vangl2KO_2.tif'
 'E10_Vangl2KO_3.tif' 'Vangl2KO_1.tif' 'Vangl2KO_2.tif' 'Vangl2KO_3.tif'
 'Vangl2KO_4.tif']


In [62]:
mask_treatment = tissue_data__experiments['Experiment'] == 'MLCKdmso-4hrs'
mask_treatment = np.where(
    tissue_data__experiments['Experiment'] == 'MLCKdmso-16hrs',
    True,
    mask_treatment
)
mask_treatment = np.where(
    tissue_data__experiments['Experiment'] == 'MLCKml7-4hrs',
    True,
    mask_treatment
)
mask_treatment = np.where(
    tissue_data__experiments['Experiment'] == 'MLCKml7-16hrs',
    True,
    mask_treatment
)
print(np.unique(tissue_data__experiments.loc[mask_treatment]['filename']))
tissue_data_errorbar(
    y='coordination_number', ylabel='HC neighbour no.',
    ylim=[3, 11],
    plot_treatment=True,
    savefig='HC_neighbours__MLCK',
    mask_treatment=mask_treatment
)

['MLCKdmso-16hrs_1.tif' 'MLCKdmso-16hrs_2.tif' 'MLCKdmso-16hrs_3.tif'
 'MLCKdmso-4hrs_1.tif' 'MLCKdmso-4hrs_2.tif' 'MLCKdmso-4hrs_3.tif'
 'MLCKml7-16hrs_1.tif' 'MLCKml7-16hrs_2.tif' 'MLCKml7-16hrs_3.tif'
 'MLCKml7-4hrs_1.tif' 'MLCKml7-4hrs_2.tif' 'MLCKml7-4hrs_3.tif']


In [63]:
tissue_data_errorbar(
    y='subordination_number', ylabel='SC neighbour number',
    ylim=[3, 11],
    figsize=[2.75, 5.9/3],
    savefig='SC_neighbourhood',
)

In [64]:
tissue_data_errorbar(
    y='subordination_number', ylabel='SC neighbour no.',
    ylim=[3, 9],
    plot_treatment=True,
    savefig='SC_neighbourhood__w_treatment',
)

In [65]:
tissue_data_errorbar(
    y='neighbor_number', ylabel='Neighbour no.',
    ylim=[4, 8],
    savefig='topology',
)

In [66]:
print(HC)

       index         filename  frame_nb  local_id_cells  area_cells  \
14383   1045   E8_25S_1_0.tif         1              35         388   
14384   1046   E8_25S_1_0.tif         1              36         457   
14424   1086   E8_25S_1_0.tif         1              76         437   
14429   1091   E8_25S_1_0.tif         1              81         396   
14433   1095   E8_25S_1_0.tif         1              85         451   
...      ...              ...       ...             ...         ...   
30873    996  E14_100_0_0.tif         0             997         580   
30874    997  E14_100_0_0.tif         0             998         290   
30879   1002  E14_100_0_0.tif         0            1003         388   
30883   1006  E14_100_0_0.tif         0            1007         302   
30889   1012  E14_100_0_0.tif         0            1013         306   

       sum_px_intensity_cells_ch1  sum_px_intensity_cells_ch2  \
14383                         255                       41577   
14384            

In [67]:
fig, ax = plt.subplots(1, 1)

sns.regplot(data=HC, x='normalized_area_cells', y='num_neighbors',
            scatter=False, color='red')
sns.kdeplot(data=HC, x='normalized_area_cells', y='num_neighbors', alpha=0.3,
            color='orange')
sns.scatterplot(
    data=HC, x='normalized_area_cells', y='num_neighbors',
    style='position', style_order=position_ordering,
    hue='stage', hue_order=stage_ordering,
    palette=palette_stages,
    alpha=0.4, size=1,
    legend=False
)

sns.lineplot(data=HC, x='HC_normalized_area', y='num_neighbors',
            **lineplot_position_stage_kwargs)

# ax.legend(bbox_to_anchor=(1.01, 1.05))

ax.set_xlim([0., 6])
ax.legend(fontsize='x-small')

plt.savefig(f"{figure_path}/num_neighbors__regplo.svg")

/tmp/ipykernel_79459/2677439691.py:16: FutureWarning: 

The `ci` parameter is deprecated. Use `errorbar='sd'` for the same effect.

  sns.lineplot(data=HC, x='HC_normalized_area', y='num_neighbors',


In [68]:
tissue_data_errorbar(
    y='area__no_norm', ylabel='Cell area [$\mu m^2$]',
    ylim=[0, None],
    savefig='tissue_size',
)

In [69]:
fig, ax = plt.subplots(1, 1)

sns.lineplot(data=HC, x='HC_normalized_area', y='shape_index',
             **lineplot_position_stage_kwargs)
_lineplot_position_stage_kwargs = lineplot_position_stage_kwargs.copy()
_lineplot_position_stage_kwargs['palette'] = 'cool'
sns.lineplot(data=SC, x='HC_normalized_area', y='shape_index', 
             **_lineplot_position_stage_kwargs)
ax.set_ylabel("Shape index")

ax.legend(bbox_to_anchor=(1.01, 1.05))

/tmp/ipykernel_79459/519145437.py:3: FutureWarning: 

The `ci` parameter is deprecated. Use `errorbar='sd'` for the same effect.

  sns.lineplot(data=HC, x='HC_normalized_area', y='shape_index',
/tmp/ipykernel_79459/519145437.py:7: FutureWarning: 

The `ci` parameter is deprecated. Use `errorbar='sd'` for the same effect.

  sns.lineplot(data=SC, x='HC_normalized_area', y='shape_index',


In [70]:
tissue_data_errorbar(
    y='hexatic_order', ylabel='Hexatic order [$\psi_6$]',
    ylim=[0, 1],
    savefig='hexatic_order',
)

In [71]:
tissue_data_errorbar(
    y='hexatic_order_corrected', ylabel='cor. Hexatic order [$\psi_6$]',
    ylim=[0, 1],
    savefig='hexatic_order_corrected',
)

In [72]:
tissue_data_errorbar(
    y='hexatic_order_corrected', ylabel='cor. Hexatic order [$\psi_6$]',
    ylim=[0, 1],
    plot_treatment=True,
    savefig='hexatic_order_corrected__w_treatment',
)

In [73]:
tissue_data_errorbar(
    y='HC_density', ylabel='HC density',
    ylim=[0, 0.4],
    figsize=[2, 2*2/3], tight_layout=False,
    savefig='HC_density',
)

In [74]:
tissue_data_errorbar(
    y='HC_density', ylabel='HC number density',
    ylim=[0, 0.4],
    plot_treatment=True,
    savefig='HC_density__w_treatment',
)

In [75]:
fig, ax = plt.subplots(1, 1)

data = cells.groupby(by=['stage', 'position', 'SI_position', 'PD_position', 'file_id'])
data = data.mean()[['HC_normalized_area', 'is_HC']]
data = data.reset_index()

data = data.loc[data['stage'] != 'E13']
data['SI_position_long'] = data['SI_position'].where(data['SI_position'] != 'S', 'Superior')
data['SI_position_long'] = data['SI_position_long'].where(data['SI_position'] != 'None', 'Mid')
data['SI_position_long'] = data['SI_position_long'].where(data['SI_position'] != 'I', 'Inferior')

data['Stage'] = data['stage']
data['S-I position'] = data['SI_position_long']
data['P-D position'] = data['PD_position']

sns.scatterplot(
  data=data,
  x='HC_normalized_area', y='is_HC',
  hue='Stage',
  hue_order=stage_ordering,
  palette=palette_stages,
  style='P-D position',
  size='S-I position',
  size_order=['Superior', 'Mid', 'Inferior'],
)

ax.set_xlim([0., 6])
ax.set_ylim([0., 0.4])
ax.set_xlabel('Normalised HC area')
ax.set_ylabel('HC density')

data = cells.groupby(by='file_id')
_area = data['normalized_area_cells'].mean()
_area_std = data['normalized_area_cells'].std()

print(data.mean())

ax.errorbar(
  x=HC.groupby(by='file_id')['normalized_area_cells'].mean(),
  y=data['is_HC'].mean(),
  xerr=HC.groupby(by='file_id')['normalized_area_cells'].std(),
  yerr=data['is_HC'].count()**0.5/data['is_HC'].count(),
  ls='', color='black', 
  linewidth=0.2, alpha=0.5,
  zorder=0
)

plt.legend(prop={'size': 'small'})

                index   frame_nb  local_id_cells   area_cells  \
file_id                                                         
0          204.566879   0.000000      205.566879   756.312102   
1          632.389785   1.000000      226.389785   899.709677   
2         1240.648177   2.000000      381.648177   880.567353   
3         2147.715054   3.000000      533.715054   610.727957   
4         3037.632353   4.000000      360.632353   480.874183   
5         3540.473958   4.743490      287.989583  3405.458333   
6         4415.403685   6.000000      346.403685   498.921273   
7          342.993255   0.000000      343.993255   675.033727   
8         2547.550699   2.295455      366.414336   893.119755   
9         1506.462867   2.000000      338.462867   712.445596   
10        3878.009146   4.456098      504.203049   508.500000   
11        3330.674812   4.000000      314.674812   987.518797   
12        3959.307400   5.000000      314.307400   571.880455   
13        5728.692967   7

In [76]:
tissue_data_errorbar(
    y='polarity__variance', ylabel='polarity variance',
    # ylim=[0, 0.4],
    savefig='polarity_variance',
)

In [77]:
tissue_data_errorbar(
    y='polarity__variance', ylabel='polarity variance',
    ylim=[0, None],
    plot_treatment=True,
    savefig='polarity_variance__w_treatment',
)

#### Hexatic order mapped

In [78]:
tissue_data_errorbar(
    x='hexatic_order', xlabel='Hexatic order [$\psi_6$]',
    y='polarity__variance', ylabel='Polarity variance',
    xlim=[0, 1.],
    ylim=[0, None],
    figsize=[2, 1.5],
    # plot_treatment=True,
    savefig='polarity_variance__vs_hexatic_order__no_correction',
)

In [79]:
tissue_data_errorbar(
    x='hexatic_order_corrected', xlabel='Hexatic order [$\psi_6$]',
    y='polarity_corrected__variance', ylabel='Polarity variance',
    xlim=[0, 1.],
    ylim=[0, 1.],
    figsize=[3, 1.6],
    # plot_treatment=True,
    savefig='polarity_variance__vs_hexatic_order',
)

In [80]:
tissue_data_errorbar(
    x='hexatic_order_corrected', xlabel='Hexatic order [$\psi_6$]',
    y='polarity_corrected__stddev_angles', ylabel='Variation of polarity\n[c.s.d. in '+r'$\degree$]',
    style=None,
    xlim=[0, 0.8],
    ylim=[0, 140],
    xticks=[0., 0.4, 0.8],
    yticks=[0, 30, 60, 90, 120],
    figsize=[2.6, 2],
    plot_errorbar=False,
    savefig='polarity_stddev__vs_hexatic_order',
)

In [81]:
mask = (tissue_data['position'] == '25I') | (tissue_data['position'] == '75I')
tissue_data_errorbar(
    x='hexatic_order_corrected', xlabel='Hexatic order [$\psi_6$]',
    y='polarity_corrected__variance', ylabel='Polarity variance',
    xlim=[0, 1.],
    ylim=[0, 1.],
    figsize=[3, 1.8],
    mask=mask,
    savefig='polarity_variance__vs_hexatic_order__25I_75I',
)

In [82]:
for treatment in tissue_data__experiments['Experiment_Series'].unique():
    tissue_data_errorbar(
        x='hexatic_order_corrected', xlabel='Hexatic order [$\psi_6$]',
        y='polarity_corrected__stddev_angles', ylabel='Variation of polarity\n[c.s.d. in '+r'$\degree$]',
        style=None,
        xlim=[0, 1.],
        ylim=[0, 140],
        yticks=[0, 30, 60, 90, 120, 140],
        figsize=[3, 1.6],
        plot_errorbar=False,
        plot_treatment=True, plot_errorbar_treatment=False,
        mask_treatment=(tissue_data__experiments['Experiment_Series']==treatment),
        savefig='polarity_stddev__vs_hexatic_order__{}'.format(treatment),
    )

/tmp/ipykernel_79459/2671978154.py:60: UserWarning: Ignoring `palette` because no `hue` variable has been assigned.
  sns.scatterplot(


In [83]:
fig, ax = plt.subplots(1, 1, figsize=[2.6, 2])
fig.tight_layout()

sns.regplot(
    data=tissue_data,
    x='hexatic_order_corrected',
    y='polarity_corrected__stddev_angles',
    # x_ci='sd', ci=None,
    color='black', scatter=False,
    line_kws={'ls': 'dashed', 'lw': linewidth},
    label='BP explants'
    # size='S-I position',
    # size_order=['Superior', 'Mid', 'Inferior']
)

data = tissue_data__experiments.copy()
data['__Experiment'] = data['Experiment']
data['Experiment'] = data['Experiment_Series']
data = data.loc[
    (data['Experiment']=='MLCK') |
    # (data['Experiment']=='MLCK_16hrs') |
    (data['Experiment']=='ROCK') |
    # (data['Experiment']=='ROCK_16hrs')
    # (data['Experiment']=='PTX') |
    (data['Experiment']=='Atoh1')
]
sns.scatterplot(
    data=data,
    x='hexatic_order_corrected',
    y='polarity_corrected__stddev_angles',
    style='is_Control',
    style_order=[True, False],
    # palette=['blue', 'red'],
    hue='Experiment',
)

order_limit = tissue_data.loc[
    (tissue_data['stage']=='E12') | (tissue_data['stage']=='E14'), 
    'hexatic_order_corrected'].min()
polarity_limit = tissue_data.loc[
    (tissue_data['stage']=='E12') | (tissue_data['stage']=='E14'), 
    'polarity_corrected__stddev_angles'].max()
ax.vlines(
    x=[order_limit],
    ymin=0, ymax=140,
    color='gray', alpha=0.7,
    lw=linewidth,
    zorder=-np.inf
)
ax.hlines(
    y=[polarity_limit],
    xmin=0, xmax=1,
    color='gray', alpha=0.7,
    lw=linewidth,
    zorder=-np.inf
)

# Create a Rectangle patch
rect = mpl.patches.Rectangle(
    (0, polarity_limit), 
    order_limit, 150,
    linewidth=0., edgecolor='none', 
    facecolor='red', alpha=0.1, 
    zorder=-np.inf
)
ax.add_patch(rect)
rect = mpl.patches.Rectangle(
    (order_limit, 0), 
    1., polarity_limit,
    linewidth=0., edgecolor='none', 
    facecolor='blue', alpha=0.1, 
    zorder=-np.inf
)
ax.add_patch(rect)

ax.set_xlim([0., 0.8])
ax.set_xticks([0., 0.4, 0.8])
ax.set_ylim([0., 140])
ax.set_yticks(range(0, 140, 30))

ax.set_xlabel('Hexatic order [$\psi_6$]')
ax.set_ylabel('Variation of polarity\n[c.s.d. in '+r'$\degree$]')

mpl.pyplot.legend(bbox_to_anchor=(1,1.025), fontsize='x-small')

plt.savefig(f"{figure_path}/area_mapped__polarity_stddev_vs_hexatic_order__inhibitions.svg")

In [84]:
fig, ax = plt.subplots(1, 1, figsize=[2.6, 2])
fig.tight_layout()

data = tissue_data__experiments.copy()
data['__Experiment'] = data['Experiment']
data['Experiment'] = data['Experiment_Series']
data = data.loc[
    (data['Experiment']=='MLCK') |
    (data['Experiment']=='ROCK') |
    (data['Experiment']=='Atoh1')
]
sns.scatterplot(
    data=data,
    x='hexatic_order_corrected',
    y='polarity_corrected__stddev_angles',
    style='is_Control',
    style_order=[True, False],
    hue='is_Control',
    hue_order=[True, False],
    palette=['blue', 'red'],
)

order_limit = tissue_data.loc[
    (tissue_data['stage']=='E12') | (tissue_data['stage']=='E14'), 
    'hexatic_order_corrected'].min()
polarity_limit = tissue_data.loc[
    (tissue_data['stage']=='E12') | (tissue_data['stage']=='E14'), 
    'polarity_corrected__stddev_angles'].max()
ax.vlines(
    x=[order_limit],
    ymin=0, ymax=140,
    color='gray', alpha=0.7,
    lw=linewidth,
    zorder=-np.inf
)
ax.hlines(
    y=[polarity_limit],
    xmin=0, xmax=1,
    color='gray', alpha=0.7,
    lw=linewidth,
    zorder=-np.inf
)

# Create a Rectangle patch
rect = mpl.patches.Rectangle(
    (0, polarity_limit), 
    order_limit, 150,
    linewidth=0., edgecolor='none', 
    facecolor='red', alpha=0.1, 
    zorder=-np.inf
)
ax.add_patch(rect)
rect = mpl.patches.Rectangle(
    (order_limit, 0), 
    1., polarity_limit,
    linewidth=0., edgecolor='none', 
    facecolor='blue', alpha=0.1, 
    zorder=-np.inf
)
ax.add_patch(rect)

ax.set_xlim([0., 0.8])
ax.set_xticks([0., 0.4, 0.8])
ax.set_ylim([0., 140])
ax.set_yticks(range(0, 140, 30))

ax.set_xlabel('Hexatic order [$\psi_6$]')
ax.set_ylabel('Variation of polarity\n[c.s.d. in '+r'$\degree$]')

mpl.pyplot.legend(bbox_to_anchor=(1,1.025), fontsize='x-small')

plt.savefig(f"{figure_path}/area_mapped__polarity_stddev_vs_hexatic_order__inhibitions__.svg")

In [85]:
fig, ax = plt.subplots(1, 1, figsize=[2.2, 1.6])

sns.regplot(
    data=tissue_data,
    x='hexatic_order_corrected',
    y='polarity_corrected__stddev_angles',
    # x_ci='sd', ci=None,
    color='black', scatter=False,
    line_kws={'ls': 'dashed', 'lw': linewidth},
    label='BP explants'
    # size='S-I position',
    # size_order=['Superior', 'Mid', 'Inferior']
)

data = tissue_data__experiments.copy()
data['__Experiment'] = data['Experiment']
data['Experiment'] = data['Experiment_Series']
data = data.loc[
    (data['Experiment']=='MLCK_16hrs') |
    (data['Experiment']=='ROCK_16hrs')
]
sns.scatterplot(
    data=data,
    x='hexatic_order_corrected',
    y='polarity_corrected__stddev_angles',
    style='is_Control',
    style_order=[True, False],
    hue='Experiment',
    palette=['tab:orange', 'tab:green'],
)

order_limit = tissue_data.loc[
    (tissue_data['stage']=='E12') | (tissue_data['stage']=='E14'), 
    'hexatic_order_corrected'].min()
polarity_limit = tissue_data.loc[
    (tissue_data['stage']=='E12') | (tissue_data['stage']=='E14'), 
    'polarity_corrected__stddev_angles'].max()
ax.vlines(
    x=[order_limit],
    ymin=0, ymax=140,
    color='gray', alpha=0.7,
    lw=linewidth,
    zorder=-np.inf
)
ax.hlines(
    y=[polarity_limit],
    xmin=0, xmax=1,
    color='gray', alpha=0.7,
    lw=linewidth,
    zorder=-np.inf
)

# Create a Rectangle patch
rect = mpl.patches.Rectangle(
    (0, polarity_limit), 
    order_limit, 150,
    linewidth=0., edgecolor='none', 
    facecolor='red', alpha=0.1, 
    zorder=-np.inf
)
ax.add_patch(rect)
rect = mpl.patches.Rectangle(
    (order_limit, 0), 
    1., polarity_limit,
    linewidth=0., edgecolor='none', 
    facecolor='blue', alpha=0.1, 
    zorder=-np.inf
)
ax.add_patch(rect)

ax.set_xlim([0., 0.8])
ax.set_ylim([0., 140])
ax.set_yticks(range(0, 140, 30))

ax.set_xlabel('Hexatic order [$\psi_6$]')
ax.set_ylabel('Variation of polarity\n[c.s.d. in '+r'$\degree$]')

mpl.pyplot.legend(bbox_to_anchor=(1,1.025), fontsize='x-small')

plt.savefig(f"{figure_path}/area_mapped__polarity_stddev_vs_hexatic_order_recovery.svg")

In [86]:
import sklearn
import sklearn.cluster

X = np.asarray(data[['hexatic_order_corrected', 'polarity_corrected__stddev_angles']])
print(X)
kmeans = sklearn.cluster.KMeans(n_clusters=4).fit(X)
print(kmeans.labels_)

fig, ax = plt.subplots(1, 1, figsize=[2.4, 1.6])

ax.scatter(X[:,0], X[:,1], c=kmeans.labels_)
ax.set_xlim([0, 1])
ax.set_ylim([0, 140])
ax.set_yticks(range(0, 140, 30))


ax.set_xlabel('Hexatic order [$\psi_6$]')
ax.set_ylabel('Variation of polarity\n[c.s.d. in '+r'$\degree$]')


[[ 0.56595154 25.54144316]
 [ 0.45687435 30.14791907]
 [ 0.4018731  60.15010485]
 [ 0.46099045 38.79735595]
 [ 0.42524707 73.66243147]
 [ 0.40099604 51.01221643]
 [ 0.43012058 61.31065749]
 [ 0.57701524 45.59554459]
 [ 0.40321158 30.28109781]
 [ 0.48606366 89.80792855]
 [ 0.39358043 62.06086439]
 [ 0.2184888  12.76356165]]
[2 2 1 0 1 0 1 0 2 3 1 2]


/home/julian/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Text(0, 0.5, 'Variation of polarity\n[c.s.d. in $\\degree$]')

## Lewis' Law

In [87]:
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)

try: 
    os.mkdir(f"{figure_path}/Lewis_law")
except OSError as error: 
    print(error)


for stage in ['E8-E12', 'E8', 'E10', 'E12', 'E14', 'all']:
    for type in ['all', 'HC', 'SC']:
        fig, ax = plt.subplots(1, 1, figsize=[5.9/3, 5.9/3])

        data = cells.loc[(cells['position'] == '25I')]

        if stage == 'E8' or stage == 'E10' or stage == 'E12' or stage == 'E14':
            data = data.loc[data['stage']==stage]
        if stage == 'E8-E12':
            data = data.loc[data['stage']!='E14']
        
        if type == 'HC':
            data = data[ data['is_HC']]
        if type == 'SC':
            data = data[~data['is_HC']]

        norm = data.groupby(by='file_id').apply(lambda grp: grp['area_cells'].mean())
        norm = norm.rename('A').reset_index()

        groubby_n = data.groupby(by=['file_id', 'stage', 'position', 'num_neighbors'])
        # A_n = groubby_n.apply(lambda grp, *, norm: print(norm.loc[norm['file_id']==grp['file_id'].iloc[0], 'A'].iloc[0]), norm=norm)
        A_n = groubby_n.apply(lambda grp, *, norm: grp['area_cells'].mean() / norm.loc[norm['file_id']==grp['file_id'].iloc[0], 'A'].iloc[0] if grp['area_cells'].count() >= 5 else np.nan, norm=norm)
        A_n = A_n.rename('A_n/A').reset_index()

        sns.scatterplot(
            data=A_n,
            x='num_neighbors',
            y='A_n/A',
            hue='stage',
            hue_order=stage_ordering,
            markers=['X'],
            style='position',
            palette=palette_stages,
        )
        sns.regplot(
            data=A_n,
            x='num_neighbors',
            y='A_n/A',
            scatter=False,
            x_ci='sd',
            label='Lin. Regression',
            line_kws={
                'color': 'red',
                'lw': linewidth
            }
        )

        _ns = np.array([i for i in range(3, 13)])
        ax.plot(
            _ns,
            (_ns - 2) / 4,
            label='Lewis Law',
            color='black',
            lw=linewidth
        )

        ax.legend(bbox_to_anchor=(1.01, 1.05))

        ax.set_xlabel('Polygon type (n)')
        ax.set_ylabel(r'$\frac{\overline{A_n}}{\overline{A}}$', rotation='horizontal')

        ax.set_xlim([2.5, 12.5])
        ax.set_xticks(range(4, 13, 2))
        ax.xaxis.set_minor_locator(MultipleLocator(1))

        if type != 'HC':
            ax.set_ylim([0,5])
            ax.set_yticks(range(0, 6))
            ax.yaxis.set_minor_locator(MultipleLocator(0.5))
        else:
            ax.set_ylim([0,3])
            ax.set_yticks(range(0, 4))
            ax.yaxis.set_minor_locator(MultipleLocator(0.5))
            

        plt.savefig(f"{figure_path}/Lewis_law/{stage}__{type}.svg")
        plt.close()

[Errno 17] File exists: '/home/julian/image_analysis/analysis/1_figures/Lewis_law'


In [88]:
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)


try: 
    os.mkdir(f"{figure_path}/Lewis_law")
except OSError as error: 
    print(error)


stage = 'E8-E12'
type = 'HC'

fig, ax = plt.subplots(1, 1, figsize=[5.9/4, 5.9/3])


data = cells.loc[(cells['position'] == '25I')]
data = data.loc[data['stage']!='E14']
data = data[~data['is_HC']]

norm = data.groupby(by='file_id').apply(lambda grp: grp['area_cells'].mean())
norm = norm.rename('A').reset_index()

groubby_n = data.groupby(by=['file_id', 'stage', 'position', 'num_neighbors'])
A_n = groubby_n.apply(lambda grp, *, norm: grp['area_cells'].mean() / norm.loc[norm['file_id']==grp['file_id'].iloc[0], 'A'].iloc[0] if grp['area_cells'].count() >= 5 else np.nan, norm=norm)
A_n = A_n.rename('A_n/A').reset_index()

sns.scatterplot(
    data=A_n,
    x='num_neighbors',
    y='A_n/A',
    markers=['X'],
    color='gray',
    label = 'SCs & lin. reg.'
)
sns.regplot(
    data=A_n,
    x='num_neighbors',
    y='A_n/A',
    scatter=False,
    x_ci='sd',
    line_kws={
        'color': 'gray',
        'lw': 2*linewidth
    }
)


data = cells.loc[(cells['position'] == '25I')]
data = data.loc[data['stage']!='E14']
data = data[ data['is_HC']]

norm = data.groupby(by='file_id').apply(lambda grp: grp['area_cells'].mean())
norm = norm.rename('A').reset_index()

groubby_n = data.groupby(by=['file_id', 'stage', 'position', 'num_neighbors'])
A_n = groubby_n.apply(lambda grp, *, norm: grp['area_cells'].mean() / norm.loc[norm['file_id']==grp['file_id'].iloc[0], 'A'].iloc[0] if grp['area_cells'].count() >= 5 else np.nan, norm=norm)
A_n = A_n.rename('A_n/A').reset_index()

sns.scatterplot(
    data=A_n,
    x='num_neighbors',
    y='A_n/A',
    markers=['X'],
    color='red',
    label = 'HCs & lin. reg.'
)
sns.regplot(
    data=A_n,
    x='num_neighbors',
    y='A_n/A',
    scatter=False,
    x_ci='sd',
    # label='Lin. Reg. HCs',
    line_kws={
        'color': 'red',
        'lw': 2*linewidth
    }
)
    


_ns = np.array([i for i in range(3, 12)])
ax.plot(
    _ns,
    (_ns - 2) / 4,
    label='Lewis Law',
    color='black',
    lw=2*linewidth
)



ax.legend(loc='upper left')
# ax.legend(bbox_to_anchor=(1.01, 1.05))

ax.set_xlabel('Polygon type (n)')
ax.set_ylabel(r'$\frac{\overline{A_n}}{\overline{A}}$', rotation='horizontal')

ax.set_xlim([2.5, 11.5])
ax.set_xticks(range(4, 12, 2))
ax.xaxis.set_minor_locator(MultipleLocator(1))

ax.set_ylim([0,3.5])
ax.set_yticks(range(0, 4))
ax.yaxis.set_minor_locator(MultipleLocator(0.5))

plt.savefig(f"{figure_path}/Lewis_law/{stage}__merge.svg")

[Errno 17] File exists: '/home/julian/image_analysis/analysis/1_figures/Lewis_law'


In [89]:
fig, ax = plt.subplots(1, 1)

data = cells.loc[(cells['stage'] == 'E10') & (cells['is_HC'])]

groubby_n = data.groupby(by=['file_id', 'stage', 'position', 'num_neighbors'])
# groubby_n.apply(lambda d: print(d[['frame_nb', 'num_neighbors', 'normalized_area_cells']]))
A_n = groubby_n.apply(lambda grp: grp['normalized_area_cells'].mean())
A_n = A_n.rename('A_n').reset_index()

sns.scatterplot(
    data=A_n,
    x='num_neighbors',
    y='A_n',
    hue='stage',
    hue_order=stage_ordering,
    style='position',
    style_order=position_ordering,
    palette=palette_stages,
)
sns.regplot(
    data=A_n,
    x='num_neighbors',
    y='A_n',
    scatter=False,
    label='lin. reg.'
)
sns.regplot(
    data=A_n,
    x='num_neighbors',
    y='A_n',
    scatter=False,
    order=2,
    label='quad. reg.'
)

ax.set_title("all cells")

_ns = np.array([i for i in range(2, 14)])
ax.plot(_ns, (_ns - 2) / 4, label='Lewis Law')

ax.plot(_ns, _ns / 6 * np.tan(math.pi / 6) / np.tan(math.pi / _ns), label='Quadr. Lewis Law')
ax.legend(bbox_to_anchor=(1.01, 1.05))


plt.savefig(f"{figure_path}/Lewis_law/E10_HC.svg")

In [90]:
print(cells_experiments['Experiment'].unique())

['Control' 'Vangl2Control' 'Vangl2KO' 'Atoh1control' 'Atoh1KO'
 'MLCKdmso-4hrs' 'MLCKdmso-16hrs' 'MLCKml7-4hrs' 'MLCKml7-16hrs'
 'ROCKcontrol-16hrs' 'ROCKcontrol-4hrs' 'ROCKy27632-16hrs'
 'ROCKy27632-4hrs' 'PTXcontrol' 'PTXinhibition']


In [91]:
_HC = HC.copy()
_HC = _HC.loc[_HC['stage'] == 'E10']
_HC = _HC.loc[_HC['position'] == '25I']

_SC = SC.copy()
_SC = _SC.loc[_SC['stage'] == 'E10']
_SC = _SC.loc[_SC['position'] == '25I']

print("Control")
print("HC norm area: {:.2f}".format(_HC['normalized_area_cells'].mean()))
print("SC norm area: {:.2f}".format(_SC['normalized_area_cells'].mean()))
print("Coordination number: {:.1f} (H: {:.2f}, S: {:.1f})".format(
    _HC['num_neighbors'].mean(),
    _HC['num_hair_neighbors'].mean(),
    _HC['num_neighbors'].mean() - _HC['num_hair_neighbors'].mean()))
print("Subordination number: {:.1f} (H: {:.1f}, S: {:.1f})".format(
    _SC['num_neighbors'].mean(),
    _SC['num_hair_neighbors'].mean(),
    _SC['num_neighbors'].mean() - _SC['num_hair_neighbors'].mean()))

_hc = cells_experiments.loc[cells_experiments['is_HC']]
_hc = _hc.loc[_hc['Experiment'] == 'MLCKml7-4hrs']
_hc = _hc.loc[_hc['stage'] == 'E10']
_hc = _hc.loc[_hc['position'] == '25I']

_sc = cells_experiments.loc[~cells_experiments['is_HC']]
_sc = _sc.loc[_sc['Experiment'] == 'MLCKml7-4hrs']
_sc = _sc.loc[_sc['stage'] == 'E10']
_sc = _sc.loc[_sc['position'] == '25I']

print("\nML-7")
print("HC norm area: {:.2f}".format(_hc['normalized_area_cells'].mean()))
print("SC norm area: {:.2f}".format(_sc['normalized_area_cells'].mean()))
print("Coordination number: {:.1f} (H: {:.2f}, S: {:.1f})".format(
    _hc['num_neighbors'].mean(),
    _hc['num_hair_neighbors'].mean(),
    _hc['num_neighbors'].mean() - _hc['num_hair_neighbors'].mean()))
print("Subordination number: {:.1f} (H: {:.1f}, S: {:.1f})".format(
    _sc['num_neighbors'].mean(),
    _sc['num_hair_neighbors'].mean(),
    _sc['num_neighbors'].mean() - _sc['num_hair_neighbors'].mean()))

Control
HC norm area: 0.88
SC norm area: 1.04
Coordination number: 5.2 (H: 0.00, S: 5.2)
Subordination number: 6.2 (H: 1.5, S: 4.7)

ML-7
HC norm area: 1.05
SC norm area: 0.97
Coordination number: 4.9 (H: 0.26, S: 4.7)
Subordination number: 6.3 (H: 2.0, S: 4.3)


In [92]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, sharey=True, sharex=True)

sns.lineplot(data=SC.loc[SC['stage'] == 'E10'], x='num_neighbors', y='normalized_area_cells', hue='stage', style='position', ci='sd', palette='cool', ax=ax1, markers=True, dashes=False, ls='', err_style='bars')
sns.lineplot(data=HC.loc[HC['stage'] == 'E10'], x='num_neighbors', y='normalized_area_cells', hue='stage', style='position', ci='sd', palette=palette_stages, ax=ax2, markers=True, dashes=False, ls='', err_style='bars')
sns.lineplot(data=cells.loc[cells['stage'] == 'E10'], x='num_neighbors', y='normalized_area_cells', hue='stage', style='position', ci='sd', palette=palette_stages, ax=ax3, markers=True, dashes=False, ls='', err_style='bars')

 
ax1.legend(bbox_to_anchor=(2.8, 1.05))
ax2.legend(bbox_to_anchor=(2.2, 1.05))


plt.savefig(f"{figure_path}/A_n.svg")

/tmp/ipykernel_79459/3481814512.py:3: FutureWarning: 

The `ci` parameter is deprecated. Use `errorbar='sd'` for the same effect.

  sns.lineplot(data=SC.loc[SC['stage'] == 'E10'], x='num_neighbors', y='normalized_area_cells', hue='stage', style='position', ci='sd', palette='cool', ax=ax1, markers=True, dashes=False, ls='', err_style='bars')
/tmp/ipykernel_79459/3481814512.py:4: FutureWarning: 

The `ci` parameter is deprecated. Use `errorbar='sd'` for the same effect.

  sns.lineplot(data=HC.loc[HC['stage'] == 'E10'], x='num_neighbors', y='normalized_area_cells', hue='stage', style='position', ci='sd', palette=palette_stages, ax=ax2, markers=True, dashes=False, ls='', err_style='bars')
/tmp/ipykernel_79459/3481814512.py:5: FutureWarning: 

The `ci` parameter is deprecated. Use `errorbar='sd'` for the same effect.

  sns.lineplot(data=cells.loc[cells['stage'] == 'E10'], x='num_neighbors', y='normalized_area_cells', hue='stage', style='position', ci='sd', palette=palette_stages, ax=ax3,

## Hexatic order

In [93]:
fig, ax = plt.subplots(1, 1, figsize=[5.9/5, 5.9/3])

points = sns.pointplot(
    data=cells,
    x='stage',
    y='hexatic_order_corrected',
    hue='position', hue_order=['25I', '75I'],
    # palette='Oranges',
    join=False,
    ci='sd',
    # capsize=.05,
    dodge=True,
    # s=2,
    errwidth=linewidth,
)
plt.setp(points.collections, sizes=[markersize])

results__hexatic_order = pd.DataFrame()
results__hexatic_order['hexatic_order'] = cells.groupby(by=['stage', 'position'])['hexatic_order_corrected'].mean()
results__hexatic_order['hexatic_order__std'] = cells.groupby(by=['stage', 'position'])['hexatic_order_corrected'].std()

print(results__hexatic_order.to_string())

leg_handles = ax.get_legend_handles_labels()[0]
ax.legend(leg_handles, ['25I', '75I'], title='')

ax.set_ylim([0, 1])

ax.set_xlabel('')
ax.set_xticklabels(['E8', 'E10', 'E12', 'E14'], fontsize=labelsize)

ax.set_ylabel('Hexatic order $\psi_6$')
ax.set_yticks([0., 0.5, 1.])


plt.savefig(f"{figure_path}/hexatic_vs_stage__explants.svg")

                hexatic_order  hexatic_order__std
stage position                                   
E10   0              0.129426            0.240505
      100            0.432462            0.257956
      25I            0.352346            0.313996
      25S            0.410273            0.267498
      50I            0.427734            0.265338
      50S            0.436800            0.266853
      75I            0.410569            0.271499
      75S            0.378363            0.268879
E12   0              0.508294            0.247676
      100            0.393695            0.233543
      25I            0.564897            0.245264
      25S            0.534323            0.247274
      50I            0.545454            0.272926
      75I            0.523138            0.260029
E14   100            0.408483            0.220119
      25I            0.648859            0.271905
      25S            0.568957            0.259316
      50I            0.645076            0.242386


/tmp/ipykernel_79459/2324627016.py:3: FutureWarning: 

The `ci` parameter is deprecated. Use `errorbar='sd'` for the same effect.

  points = sns.pointplot(


In [94]:
fig, ax = plt.subplots(1, 1, figsize=[5.9/4, 5.9/3])

points = sns.pointplot(
    data=cells.loc[cells['position']=='25I'],
    x='stage',
    y='hexatic_order_corrected',
    # hue='position', hue_order=['25I'],
    color='red',
    join=False,
    errorbar='sd',
    # capsize=.05,
    # dodge=True,
    # s=2,
    errwidth=linewidth,
    markers='.',
)
plt.setp(points.collections, sizes=[markersize])

results__hexatic_order = pd.DataFrame()
results__hexatic_order['hexatic_order'] = cells.groupby(by=['stage', 'position'])['hexatic_order_corrected'].mean()
results__hexatic_order['hexatic_order__std'] = cells.groupby(by=['stage', 'position'])['hexatic_order_corrected'].std()

print(results__hexatic_order.to_string())

# leg_handles = ax.get_legend_handles_labels()[0]
# ax.legend(leg_handles, ['25I', '75I'], title='')

ax.set_ylim([0, 1])

ax.set_xlabel('')
ax.set_xticklabels(['E8', 'E10', 'E12', 'E14'], fontsize=labelsize)

ax.set_ylabel('Hexatic order $\psi_6$')
ax.set_yticks([0., 0.5, 1.])


plt.savefig(f"{figure_path}/hexatic_vs_stage__25I.svg")

                hexatic_order  hexatic_order__std
stage position                                   
E10   0              0.129426            0.240505
      100            0.432462            0.257956
      25I            0.352346            0.313996
      25S            0.410273            0.267498
      50I            0.427734            0.265338
      50S            0.436800            0.266853
      75I            0.410569            0.271499
      75S            0.378363            0.268879
E12   0              0.508294            0.247676
      100            0.393695            0.233543
      25I            0.564897            0.245264
      25S            0.534323            0.247274
      50I            0.545454            0.272926
      75I            0.523138            0.260029
E14   100            0.408483            0.220119
      25I            0.648859            0.271905
      25S            0.568957            0.259316
      50I            0.645076            0.242386


## Spatial gradients

In [95]:
fig, ax = plt.subplots(1, 1, figsize=[2.3, 1.5])
fig.tight_layout()

points = sns.pointplot(
    data=cells.loc[cells['stage'] == 'E14'],
    x='position',
    order=['25I', '25S', '50I', '50S', '75I', '75S', '100'],
    y='area',
    hue='is_HC',
    hue_order=[False, True],
    palette=['gray', 'red'],
    join=False,
    errorbar='sd',
    capsize=.05,
    dodge=True,
    # s=2,
    errwidth=linewidth,
    markers='.',
)
plt.setp(points.collections, sizes=[markersize])

ax.set_ylim([0, 90])
ax.set_yticks([0, 30, 60, 90])
ax.set_xlim([-0.5, 6.5])

ax.set_ylabel('Cell area [$\mu m^2$]')

ax.set_xlabel('')
ax.set_xticklabels(['25I', '25S', '50I', '50S', '75I', '75S', '100'], fontsize=labelsize)

leg_handles = ax.get_legend_handles_labels()[0]
ax.legend(leg_handles, ['SC', 'HC'], title='')

plt.savefig(f"{figure_path}/HC_area__E14__no_norm.svg",format='svg')

In [96]:
fig, ax = plt.subplots(1, 1, figsize=[2.3, 1.5])
fig.tight_layout()

points = sns.pointplot(
    data=cells.loc[cells['stage'] == 'E14'],
    x='position',
    order=['25I', '25S', '50I', '50S', '75I', '75S', '100'],
    y='num_neighbors',
    hue='is_HC',
    hue_order=[False, True],
    palette=['gray', 'red'],
    join=False,
    errorbar='sd',
    capsize=.05,
    dodge=True,
    # s=2,
    errwidth=linewidth,
    markers='.',
)
plt.setp(points.collections, sizes=[markersize])

ax.set_ylim([3, 12])
ax.hlines(y=[6], xmin=-0.5, xmax=6.5, color='black', ls=':', linewidth=linewidth)
ax.set_yticks([3, 6, 9, 12])
ax.set_xlim([-0.5, 6.5])

ax.set_ylabel('Neighbour number')

ax.set_xlabel('')
ax.set_xticklabels(['25I', '25S', '50I', '50S', '75I', '75S', '100'], fontsize=labelsize)

leg_handles = ax.get_legend_handles_labels()[0]
ax.legend(leg_handles, ['SC', 'HC'], title='')

plt.savefig(f"{figure_path}/Neighbours__E14.svg",format='svg')

In [97]:
fig, ax = plt.subplots(1, 1, figsize=[2.3, 1.5])
fig.tight_layout()

points = sns.pointplot(
    data=cells.loc[cells['stage'] == 'E14'],
    x='position',
    order=['25I', '25S', '50I', '50S', '75I', '75S', '100'],
    y='hexatic_order_corrected',
    hue='is_HC',
    hue_order=[True],
    palette=['red'],
    join=False,
    errorbar='sd',
    capsize=.05,
    dodge=True,
    # s=2,
    errwidth=linewidth,
    markers='.',
)
plt.setp(points.collections, sizes=[markersize])

ax.set_ylim([0, 1.])
ax.set_yticks([0., 0.5, 1.])
ax.set_xlim([-0.5, 6.5])

ax.set_ylabel('Hexatic order $\psi_6$')

ax.set_xlabel('')
ax.set_xticklabels(
    ['25I', '25S', '50I', '50S', '75I', '75S', '100'],
    fontsize=labelsize
)

leg_handles = ax.get_legend_handles_labels()[0]
ax.legend(leg_handles, ['HC'], title='')

plt.savefig(f"{figure_path}/Hexatic_order__E14.svg",format='svg')

In [98]:
fig, ax = plt.subplots(1, 1, figsize=[2.1, 1.5])
fig.tight_layout()

points = sns.pointplot(
    data=tissue_data.loc[tissue_data['stage']=='E14'],
    x='position',
    order=['25I', '25S', '50I', '50S', '75I', '75S', '100'],
    y='HC_density',
    color='black',
    join=False,
    # color='black',
    errorbar='sd',
    capsize=.05,
    dodge=True,
    # s=2,
    errwidth=linewidth,
    markers='.',
)
plt.setp(points.collections, sizes=[markersize])

ax.set_ylim([0, 0.4])
ax.set_xlim([-0.5, 6.5])

ax.set_ylabel('HC number density')

ax.set_xlabel('')
ax.set_xticklabels(
    ['25I', '25S', '50I', '50S', '75I', '75S', '100'], 
    fontsize=labelsize
)

# ax.legend()

plt.savefig(f"{figure_path}/HC_density__E14.svg",format='svg')

In [99]:
fig, ax = plt.subplots(1, 1, figsize=[2.2, 1.6])
fig.tight_layout()

points = sns.pointplot(
    data=tissue_data.loc[
        (tissue_data['stage']=='E10') | (tissue_data['stage']=='E14')
    ],
    x='position',
    order=['25I', '25S', '50I', '50S', '75I', '75S', '100'],
    y='HC_density',
    hue='stage',
    hue_order=['E10', 'E14'],
    palette=['gray', 'black'],
    join=False,
    # color='black',
    errorbar='sd',
    capsize=.05,
    dodge=True,
    # s=2,
    errwidth=linewidth,
    markers='.',
)
plt.setp(points.collections, sizes=[markersize])

ax.set_ylim([0, 0.4])
ax.set_xlim([-0.5, 6.5])

ax.set_ylabel('HC number density')

ax.set_xlabel('')
ax.set_xticklabels(
    ['25I', '25S', '50I', '50S', '75I', '75S', '100'], 
    fontsize=labelsize
)

plt.savefig(f"{figure_path}/HC_density__E10_E14.svg",format='svg')

In [100]:
fig, ax = plt.subplots(1, 1, figsize=[5.9/3, 5.9/3])

points = sns.pointplot(
    data=tissue_data.loc[tissue_data['stage'] == 'E14'],
    x='position',
    order=['25I', '25S', '50I', '50S', '75I', '75S', '100'],
    y='number_ratio',
    join=False,
    errorbar='sd',
    capsize=.05,
    dodge=True,
    # s=2,
    errwidth=linewidth,
    markers='.',
)
plt.setp(points.collections, sizes=[markersize])

ax.set_ylim([0.2, 0.8])
ax.hlines(y=[0.5], xmin=-0.5, xmax=6.5, colors=['black'], linewidth=linewidth)
ax.set_xlim([-0.5, 6.5])

ax.set_ylabel('HC / SC number ratio')

ax.set_xlabel('')
ax.set_xticklabels(['25I', '25S', '50I', '50S', '75I', '75S', '100'], fontsize=labelsize)

leg_handles = ax.get_legend_handles_labels()[0]
ax.legend(leg_handles, ['SC', 'HC'], title='')

plt.savefig(f"{figure_path}/HC_ratio__E14.svg",format='svg')

## Polarity

In [101]:
fig, ax = plt.subplots(1, 1)


sns.pointplot(
    data=tissue_data,
    x='Stage',
    y='polarity_corrected__variance',
    hue='position', hue_order=['25I', '75I'],
    # palette='Oranges',
    join=False,
    ci='sd',
    # err_style='bars',
    capsize=.05,
    dodge=True,
)

# ax.set_xlim([0, 1])
ax.set_ylim([0, 1])

ax.set_ylabel('Circular variance of polarity (corr.)')



plt.legend(prop={'size': 'small'})

plt.savefig(f"{figure_path}/polarity_variance_vs_stage__explants.svg")

/tmp/ipykernel_79459/3587476627.py:4: FutureWarning: 

The `ci` parameter is deprecated. Use `errorbar='sd'` for the same effect.

  sns.pointplot(


In [102]:
results = pd.DataFrame()

_data = cells.loc[cells['is_HC']].copy()
_data = _data.loc[(_data['aligned_with_PD_axis']) | (_data['stage'] == 'E14')].copy()
_data = _data.loc[(_data['position'] == '25I') | (_data['position'] == '75I')].copy()

fig, axes = plt.subplots(
    1, 4,
    sharex=True, sharey=True,
    subplot_kw=dict(projection='polar'),
    figsize=[7., 3],
)
plt.subplots_adjust(hspace=0.)
(ax1, ax2, ax3, ax4) = axes
for i, (stage, ax) in enumerate(zip(['E8', 'E10', 'E12', 'E14'], axes)):
    plt.sca(ax)
    data = _data.loc[_data['stage'] == stage].copy()
    
    palette = ['tab:blue', 'tab:orange']
    __data = data
    tmp = data
    tmp['cilium_phi_corrected'] = data['cilium_phi_corrected'] + 2*math.pi
    __data = pd.concat([__data, tmp])
    tmp['cilium_phi_corrected'] = data['cilium_phi_corrected'] - 2*math.pi
    __data = __data.loc[__data['cilium_phi_corrected'] > - 1.2 * math.pi]
    __data = __data.loc[__data['cilium_phi_corrected'] <   1.2 * math.pi]
    __data = pd.concat([__data, tmp])
    sns.kdeplot(
        data=__data.reset_index(),
        x='cilium_phi_corrected',
        y='cilium_rho_corrected',
        hue='position',
        hue_order=['25I', '75I'],
        palette=palette,
        # bw_method=0.1,
        common_norm=False,
        fill=True,
        levels=3,
        thresh=0.2,
        alpha=0.4,
        zorder=10,
    )
    
    sns.scatterplot(
        data=data,
        x='cilium_phi_corrected',
        y='cilium_rho_corrected',
        hue='position',
        hue_order=['25I', '75I'],
        palette=palette,
        # style='file_id',
        alpha=0.75,
        s=3,
        edgecolors=None,
        ax=ax,
        zorder=11,
    )

    # sns.histplot(
    #     data=data,
    #     x='cilium_phi_corrected',
    #     weights='cilium_rho_corrected',
    #     hue='position',
    #     hue_order=['25I', '75I'],
    #     multiple='stack',
    #     bins=16,
    #     binrange=[-math.pi, math.pi],
    #     stat='probability',
    #     zorder=20,
    # )
    
    quiver_x = []
    quiver_y = []
    quiver_u = []
    quiver_v = []
    for pos in np.unique(data['position']):
        d = data.loc[data['position'] == pos]
        mean = circular_mean(d[['cilium_DX_corrected', 'cilium_DY_corrected']], eulerian=True)
        mean_length = circular_mean_length(d[['cilium_DX_corrected', 'cilium_DY_corrected']], eulerian=True, normalize=False)

        quiver_x.append(0)
        quiver_y.append(0)
        quiver_u.append(mean)
        quiver_v.append(mean_length)

    ax.quiver(
        quiver_x, quiver_y, quiver_u, quiver_v,
        facecolor=palette, edgecolor='black', linewidth=linewidth,
        scale=1, angles='xy', scale_units='xy',
        zorder=12
    )

    ax.invert_xaxis()
    ax.set_xlim(-math.pi, math.pi)
    ax.set_xticks([-math.pi, -math.pi / 2, 0, math.pi / 2])
    ax.set_xticklabels(['P', 'I', 'D', 'S'])
    ax.tick_params(axis='x', which='major', pad=-5)

    ax.set_ylim(0, 1.2)
    ax.set_yticks([0.5, 1.])

    ax.set_xlabel('')
    ax.set_ylabel('')

    ax.grid(zorder=-100)

    if i == len(axes) - 1:
        mpl.pyplot.legend(bbox_to_anchor=(1.255,1.25))
    else:
        ax.get_legend().remove()

plt.savefig(f"{figure_path}/polarity.svg")
        

# human readable information

grp = ['stage', 'position', 'file_id']
polarity_by_sample = _data.groupby(by=grp)[['cilium_DX_corrected', 'cilium_DY_corrected']].mean()
polarity_by_sample['circular_mean'] = _data.groupby(by=grp)[['cilium_DX_corrected', 'cilium_DY_corrected']].apply(circular_mean, eulerian=True)
polarity_by_sample['circular_mean_length'] = _data.groupby(by=grp)[['cilium_DX_corrected', 'cilium_DY_corrected']].apply(circular_mean_length, eulerian=True, normalize=False)
polarity_by_sample['circular_stddev'] = _data.groupby(by=grp)[['cilium_DX_corrected', 'cilium_DY_corrected']].apply(circular_stddev, eulerian=True)
polarity_by_sample['datapoints'] = _data.groupby(by=grp)['cilium_DX_corrected'].apply(lambda d: d.count())

polarity_by_sample['circular_mean__angles'] = (polarity_by_sample['circular_mean']/math.pi * 180 + 360) % 360
polarity_by_sample['circular_stddev__angles'] = (polarity_by_sample['circular_stddev']/math.pi * 180 + 360) % 360
print(polarity_by_sample.to_string())

grp = ['stage', 'position']
polarity_by_position = _data.groupby(by=grp)[['cilium_DX_corrected', 'cilium_DY_corrected']].mean()
polarity_by_position['circular_mean'] = _data.groupby(by=grp)[['cilium_DX_corrected', 'cilium_DY_corrected']].apply(circular_mean, eulerian=True)
polarity_by_position['circular_mean_length'] = _data.groupby(by=grp)[['cilium_DX_corrected', 'cilium_DY_corrected']].apply(circular_mean_length, eulerian=True, normalize=False)
polarity_by_position['circular_stddev'] = _data.groupby(by=grp)[['cilium_DX_corrected', 'cilium_DY_corrected']].apply(circular_stddev, eulerian=True)
polarity_by_position['datapoints'] = _data.groupby(by=grp)['cilium_DX_corrected'].apply(lambda d: d.count())

polarity_by_position['circular_mean__angles'] = (polarity_by_position['circular_mean']/math.pi * 180 + 360) % 360
polarity_by_position['circular_stddev__angles'] = (polarity_by_position['circular_stddev']/math.pi * 180 + 360) % 360
print(polarity_by_position.to_string())

grp = ['stage']
polarity_by_stage = _data.groupby(by=grp)[['cilium_DX_corrected', 'cilium_DY_corrected']].mean()
polarity_by_stage['circular_mean'] = _data.groupby(by=grp)[['cilium_DX_corrected', 'cilium_DY_corrected']].apply(circular_mean, eulerian=True)
polarity_by_stage['circular_mean_length'] = _data.groupby(by=grp)[['cilium_DX_corrected', 'cilium_DY_corrected']].apply(circular_mean_length, eulerian=True, normalize=False)
polarity_by_stage['circular_stddev'] = _data.groupby(by=grp)[['cilium_DX_corrected', 'cilium_DY_corrected']].apply(circular_stddev, eulerian=True)
polarity_by_stage['datapoints'] = _data.groupby(by=grp)['cilium_DX_corrected'].apply(lambda d: d.count())

polarity_by_stage['circular_mean__angles'] = (polarity_by_stage['circular_mean']/math.pi * 180 + 360) % 360
polarity_by_stage['circular_stddev__angles'] = (polarity_by_stage['circular_stddev']/math.pi * 180 + 360) % 360
print(polarity_by_stage.to_string())


                        cilium_DX_corrected  cilium_DY_corrected  circular_mean  circular_mean_length  circular_stddev  datapoints  circular_mean__angles  circular_stddev__angles
stage position file_id                                                                                                                                                            
E10   25I      7                  -0.195670            -0.206414      -2.329480              0.284417         0.779262         129             226.530649                44.648423
               8                  -0.265017            -0.211662      -2.467662              0.339168         0.674124          96             218.613378                38.624455
      75I      11                  0.251111             0.043626       0.172015              0.254873         0.807942          97               9.855721                46.291672
               12                  0.177554             0.032502       0.181050              0.180504    

In [103]:
!pip install pycircstat
# !pip install nose
import pycircstat

!pip install hotelling
from hotelling.stats import hotelling_t2

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [104]:
treatments = dict({
    'MLCK': [
        'MLCK',
        'MLCK_16hrs',
    ],
    'ROCK': [
        'ROCK',
        'ROCK_16hrs',
    ],
    'PTX': [
        'PTX'
    ],
    'Atoh1': [
        'Atoh1'
    ],
    'Vangl2': [
        'Vangl2'
    ]
})

figsizes=dict({
    'ROCK': 1.6,
    'Atoh1': 1.8,
})
# for Mode in ['aligned']:
for Mode in ['original', 'aligned', 'by_file', 'centred']:
    if Mode == 'by_file':
        mean_by_file=True
    else:
        mean_by_file=False

    for treatment, items in treatments.items():
        if treatment in figsizes:
            figsize=figsizes[treatment]
        else:
            figsize=1.4
        if len(items) > 1:
            fig, axes = plt.subplots(
                1, len(items),
                sharex=True, sharey=True,
                subplot_kw=dict(projection='polar'),
                figsize=[len(items)*figsize, figsize],
            )

            plt.subplots_adjust(hspace=0.)
        elif len(items) == 1:
            fig, ax = plt.subplots(
                1, 1,
                sharex=True, sharey=True,
                subplot_kw=dict(projection='polar'),
                figsize=[figsize, figsize],
            )
            axes = [ax]

        for i, ax in enumerate(axes):
            plt.sca(ax)
            data = cells_experiments.loc[cells_experiments['Experiment_Series']==items[i]].copy()
            data = data.sort_values(by='is_Control')

            N = data.groupby(by='is_Control')['cilium_phi_corrected'].count()
            markersize=3
            markersize_control=markersize
            markersize_inhibition=markersize
            if N.iloc[0] > 5 * N.iloc[1]:
                markersize_control *= 2
            elif 5 * N.iloc[0] > N.iloc[1]:
                markersize_inhibition *= 2

            # correct 
            if Mode == 'aligned':
                phi_column='cilium_phi_corrected__aligned'
                rho_column='cilium_rho_corrected'
                dx_column='cilium_DX_corrected__aligned'
                dy_column='cilium_DY_corrected__aligned'
            elif Mode == 'centred':
                phi_column='cilium_phi_corrected__centred'
                rho_column='cilium_rho_corrected'
                dx_column='cilium_DX_corrected__centred'
                dy_column='cilium_DY_corrected__centred'

            else:
                phi_column='cilium_phi_corrected'
                rho_column='cilium_rho_corrected'
                dx_column='cilium_DX_corrected'
                dy_column='cilium_DY_corrected'

            palette = ['tab:blue', 'tab:red'] # ['navy', 'orange']
            hue_order=[False, True]
            style=None
            if Mode == 'by_file':
                style='file_id'
            sns.scatterplot(
                data=data,
                x=phi_column,
                y=rho_column,
                hue='is_Control',
                hue_order=hue_order,
                size='is_Control',
                size_order=[True, False],
                sizes=[markersize_control, markersize_inhibition],
                style=style,
                palette=palette,
                alpha=0.75,
                # s=3,
                edgecolors=None,
                ax=ax,
                zorder=11,
            )
            __data = data.copy()

            # if Mode == 'aligned':
            if Mode == 'centred':
                print(
                    f"{items[i]} ({Mode}): p = %.2e"%pycircstat.watson_williams(
                        data.loc[data['is_Control']==True, phi_column].dropna(),
                        data.loc[data['is_Control']==False, phi_column].dropna()
                    )[0]
                )
                print(
                    f"{items[i]} ({Mode}): hotelling p = %.2e"%hotelling_t2(
                        data.loc[data['is_Control']==True, [dx_column, dy_column]].dropna(),
                        data.loc[data['is_Control']==False, [dx_column, dy_column]].dropna()
                    )[2]
                )
                print(
                    f"Uniformity Control: p = %.2e"%pycircstat.rayleigh(
                        data.loc[data['is_Control']==True, phi_column].dropna(),
                    )[0]
                )
                print(
                    f"Uniformity treatment: p = %.2e"%pycircstat.rayleigh(
                        data.loc[data['is_Control']==False, phi_column].dropna()
                    )[0]
                )
                print()

            tmp = data.loc[data[phi_column] < -1.2 * math.pi].copy()
            tmp[phi_column] = tmp[phi_column] + 2*math.pi
            __data = pd.concat([__data, tmp])

            tmp = data.loc[data[phi_column] > 1.2 * math.pi].copy()
            tmp[phi_column] = tmp[phi_column] - 2*math.pi
            __data = pd.concat([__data, tmp])

            sns.kdeplot(
                data=__data.reset_index(),
                x=phi_column,
                y=rho_column,
                hue='is_Control',
                hue_order=hue_order,
                palette=palette,
                fill=True,
                common_norm=False,
                levels=3,
                thresh=.2,
                alpha=0.4,
                zorder=10,
            )
            
            quiver_x = []
            quiver_y = []
            quiver_u = []
            quiver_v = []
            color = []
            for j, is_control in enumerate(hue_order):
                d = data.loc[data['is_Control'] == is_control]

                # if treatment == 'Vangl2':
                if mean_by_file:
                    mean_phi = d.groupby(by='file_id')[[dx_column, dy_column]].apply(circular_mean, eulerian=True)
                    mean_rho = d.groupby(by='file_id')[[dx_column, dy_column]].apply(circular_mean_length, eulerian=True, normalize=False)

                else:
                    mean_phi = [circular_mean(d[[dx_column, dy_column]], eulerian=True)]
                    mean_rho = [circular_mean_length(d[[dx_column, dy_column]], eulerian=True, normalize=False)]
                
                for phi, rho in zip(mean_phi, mean_rho):
                    quiver_x.append(0)
                    quiver_y.append(0)
                    quiver_u.append(phi)
                    quiver_v.append(rho)
                    color.append(palette[j])

            ax.quiver(
                quiver_x, quiver_y, quiver_u, quiver_v,
                facecolor=color, edgecolor='black', linewidth=linewidth,
                scale=1, angles='xy', scale_units='xy',
                zorder=12
            )

            ax.set_xlim(-math.pi, math.pi)
            ax.set_xticks([-math.pi, -math.pi / 2, 0, math.pi / 2])
            ax.set_xticklabels(['P', 'I', 'D', 'S'])
            ax.tick_params(axis='x', which='major', pad=-5)

            ax.set_ylim(0, 1.2)
            ax.set_yticks([0.5, 1.])

            ax.set_xlabel('')
            ax.set_ylabel('')

            if i == len(axes) - 1:
                leg_handles = ax.get_legend_handles_labels()[0]
                mpl.pyplot.legend(
                    leg_handles[::-1],
                    ['Control', f'{treatment}'],
                    title='',
                    fontsize=ticksize,
                    bbox_to_anchor=(1.05,1.05),
                    markerscale=1
                )
            else:
                ax.get_legend().remove()

        plt.savefig(f"{figure_path}/polarity__{treatment}__{Mode}.svg")

MLCK (centred): p = 6.87e-01
MLCK (centred): hotelling p = 5.17e-29
Uniformity Control: p = 6.22e-54
Uniformity treatment: p = 1.44e-04

MLCK_16hrs (centred): p = 4.52e-01
MLCK_16hrs (centred): hotelling p = 2.05e-02
Uniformity Control: p = 8.00e-71
Uniformity treatment: p = 1.50e-28

ROCK (centred): p = 8.80e-02
ROCK (centred): hotelling p = 2.63e-14
Uniformity Control: p = 1.09e-48
Uniformity treatment: p = 2.54e-02

ROCK_16hrs (centred): p = 9.22e-01
ROCK_16hrs (centred): hotelling p = 1.73e-02
Uniformity Control: p = 1.10e-39
Uniformity treatment: p = 3.96e-07

PTX (centred): p = 9.84e-01
PTX (centred): hotelling p = 4.92e-07
Uniformity Control: p = 1.16e-68
Uniformity treatment: p = 5.11e-10

Atoh1 (centred): p = 6.06e-01
Atoh1 (centred): hotelling p = 3.09e-41
Uniformity Control: p = 1.60e-110
Uniformity treatment: p = 2.41e-09

Vangl2 (centred): p = 5.70e-01
Vangl2 (centred): hotelling p = 2.97e-05
Uniformity Control: p = 5.07e-45
Uniformity treatment: p = 1.70e-63



## Atoh1 density

In [105]:

fix, ax = plt.subplots(1, 1, figsize=[1.2, 1.7])

sns.violinplot(
    data=tissue_data__experiments,
    x='Treatment',
    order=['Atoh1control', 'Atoh1KO'],
    palette=['tab:red', 'tab:blue'],
    y='HC_density',
    inner='point',
    linewidth=linewidth
)

box_pairs=[
    ('Atoh1control', 'Atoh1KO'),
]

annotator = Annotator(
    ax,
    box_pairs,
    data=tissue_data__experiments,
    x='Treatment',
    order=['Atoh1control', 'Atoh1KO'],
    y='HC_density',
    split=False,
)
text_format='star'
annotator.configure(
    test='Mann-Whitney',
    line_width=0.25,
    loc='outside',
    text_format=text_format
)
annotator.apply_and_annotate()


ax.set_xlabel('')
ax.set_xticklabels(['Control', 'Atoh1-KO'], rotation=30, ha='right', rotation_mode='anchor')

ax.set_ylabel('HC density')
ax.set_ylim(0, 0.4)
ax.set_yticks([0, 0.2, 0.4])

plt.savefig(f"{figure_path}/HC_density_Atoh1.svg")

p-value annotation legend:
      ns: p <= 1.00e+00
       *: 1.00e-02 < p <= 5.00e-02
      **: 1.00e-03 < p <= 1.00e-02
     ***: 1.00e-04 < p <= 1.00e-03
    ****: p <= 1.00e-04

Atoh1control vs. Atoh1KO: Mann-Whitney-Wilcoxon test two-sided, P_val:2.857e-02 U_stat=1.600e+01


# Supplementary information

In [106]:
print("Available datasets: ")
for set in cells.columns:
  print("  {}".format(set))

Available datasets: 
  index
  filename
  frame_nb
  local_id_cells
  area_cells
  sum_px_intensity_cells_ch1
  sum_px_intensity_cells_ch2
  sum_px_intensity_cells_ch3
  sum_px_intensity_cells_12_bits
  first_pixel_x_cells
  first_pixel_y_cells
  center_x_cells
  center_y_cells
  s1_stretch_cells
  s2_stretch_cells
  s0_stretch_cells
  orientation_cells
  vx_coords_cells
  nb_of_perimeter_pixels
  perimeter_length
  sum_px_intensity_perimeter_ch1
  sum_px_intensity_perimeter_ch2
  sum_px_intensity_perimeter_ch3
  sum_px_intensity_12_perimeter_bits
  p1_polarity_ch1
  p2_polarity_ch1
  p1_polarity_ch2
  p2_polarity_ch2
  p1_polarity_ch3
  p2_polarity_ch3
  p1_polarity_12bits
  p2_polarity_12bits
  p1_polarity_ch1_normalized
  p2_polarity_ch1_normalized
  p1_polarity_ch2_normalized
  p2_polarity_ch2_normalized
  p1_polarity_ch3_normalized
  p2_polarity_ch3_normalized
  p1_polarity_12bits_normalized
  p2_polarity_12bits_normalized
  deprecated_v1_polarity_ch1
  deprecated_v2_polarity_ch1


In [107]:
print("Available datasets in bonds: ")
for set in bonds.columns:
  print("  {}".format(set))

Available datasets in bonds: 
  index
  filename
  frame_nb
  local_id_bonds
  bond_size_in_px
  bond_length_in_px
  bond_orientation
  vx_1_x
  vx_1_y
  vx_2_x
  vx_2_y
  sum_px_int_vertices_excluded_ch1
  sum_px_int_vertices_excluded_ch2
  sum_px_int_vertices_excluded_ch3
  sum_px_int_vertices_excluded_12bits
  sum_px_int_vertices_included_ch1
  sum_px_int_vertices_included_ch2
  sum_px_int_vertices_included_ch3
  sum_px_int_vertices_included_12bits
  first_pixel_x_bonds
  first_pixel_y_bonds
  cell_id_around_bond1
  cell_id_around_bond2
  Unnamed: 22
  filepath
  stage
  position
  sample_id
  crop_id
  file_id
  global_id_bonds
  global_cell_id_around_bond1
  global_cell_id_around_bond2
  aligned_with_PD_axis
  PD_position
  SI_position
  resolution_x
  resolution_y
  resolution_scale
  type
  normalized_bond_length
  ppMLC_foci_X
  ppMLC_foci_Y
  ppMLC_foci_distance
  ppMLC_foci_Size
  ppMLC_foci_Max
  ppMLC_foci_Total
  Experiment
  SI_position_long
  Stage
  S-I position
  P-D p